In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# API Spotify

Utilizamos la API de Spotify para obtener canciones con sus nombres, sus artistas y sus caracteristicas musicales.

In [ ]:
# Instalar Spotipy
!pip install spotipy

import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Configuración de credenciales
client_id = "c1322b062dfb44f7b1b6d7db72dabef2"       # Reemplaza con tu Client ID
client_secret = "006c74d33040470587a9ca2b3dddc84d"  # Reemplaza con tu Client Secret
redirect_uri = "https://example.com/callback"  # Reemplaza con la URI configurada en Spotify Dashboard

# Configurar autenticación
scope = "user-library-read"  # Define el alcance según lo que necesites
sp_oauth = SpotifyOAuth(client_id, client_secret, redirect_uri, scope=scope)

# Generar la URL para autenticación
auth_url = sp_oauth.get_authorize_url()
print(f"Por favor, autoriza la aplicación visitando este enlace:\n{auth_url}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.7 MB/s eta 0:00:00
Por favor, autoriza la aplicación visitando este enlace:
https://accounts.spotify.com/authorize?client_id=c1322b062dfb44f7b1b6d7db72dabef2&response_type=code&redirect_uri=https%3A%2F%2Fexample.com%2Fcallback&scope=user-library-read


###Token de acceso

In [ ]:
# Pega el código obtenido
code = 'AQBmHFGBW0QBxXNF7sYUuHBHG2nAj2T5lJlXsE2r_UFcmufMAbHa7LBIj5k5siAlr8fBR1_av-ovupn6Vrg8Dmt0G97tvq7OPpZrUaxMFs6jVU2OyDEbpD-qG4iJm5mKQOtKeuTIkUoq7HeewKkWTO36QroEG5YnKQgiVUHG-UxLpAWDiAOdCPRGH3t5Q49Hz_5x'

# Intercambiar el código por un token de acceso
token_info = sp_oauth.get_access_token(code)
access_token = token_info["access_token"]
print("Token de acceso:", access_token)

# Conectar a la API usando el token
sp = spotipy.Spotify(auth=access_token)

Token de acceso: BQB6RUDsnCp6wJ6JB7lq3bMi2ZiuudPxDg7x7SX94s9WOO8InTppmIiSXM5jGLmxcofSvjkzTT5bHtVBe2KEVgp-GicN7O-MJ1bSjLK4IEZqT-_rEpSKfVR4dm1RJ8eTWhdeUAXFSDFlSDG9xx_MP4b-YiQwnQEhYq_eXZ_G8w7GDw_I7N4fnr0J5WwNMOGV8owe2gFey_ya9w5rjPj6UnDBOGQ


<ipython-input-3-fd431e808656>:5: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token(code)


###**Busqueda de canciones por playlist**

###Funciones

In [ ]:
import requests
import json
import time

def search_playlist(query, access_token):
    """Busca una playlist en Spotify por su nombre y devuelve el ID de la playlist."""
    url = f"https://api.spotify.com/v1/search?q={query}&type=playlist&limit=1"  # limit=1 para solo buscar la primera coincidencia
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        search_results = response.json()
        playlists = search_results["playlists"]["items"]
        if playlists:
            # Si hay playlists que coinciden, devolvemos el ID de la primera
            playlist = playlists[0]
            return {"id": playlist["id"], "name": playlist["name"]}
        else:
            print(f"No se encontraron playlists con el nombre '{query}'.")
            return None
    else:
        print("Error al buscar playlists:", response.json())
        return None

def get_track_ids_from_playlist(playlist_id, access_token, limit=50):
    """Obtiene los IDs de las canciones de una playlist de Spotify."""
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = {"Authorization": f"Bearer {access_token}"}
    track_ids = []

    while url:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            # Extraer los track IDs de las canciones
            track_ids.extend([item["track"]["id"] for item in data["items"]]) #if item["track"]])

            # Verificar si hay más canciones en la siguiente página
            url = data["next"]
        else:
            print(f"Error al obtener canciones de la playlist {playlist_id}: {response.json()}")
            break

    return track_ids


def get_artist_genre(artist_id, access_token):
    """Obtiene el género de un artista a partir de su ID."""
    url = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        genres = data.get("genres", [])
        return genres[0] if genres else "Unknown"  # Devuelve el primer género o "Unknown"
    else:
        print(f"Error al obtener género del artista {artist_id}: {response.json()}")
        return "Unknown"

def get_audio_features(track_id, access_token):
    """Obtiene las características de una canción a partir de su ID."""
    url = f"https://api.spotify.com/v1/audio-features/{track_id}"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error al obtener características de la canción {track_id}: {response.json()}")
        return None

def get_track_details(track_id, access_token):
    """Obtiene detalles (nombre, artista, género y características musicales) de una canción a partir de su ID."""
    url = f"https://api.spotify.com/v1/tracks/{track_id}"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        name = data["name"]
        artist = data["artists"][0]["name"]
        artist_id = data["artists"][0]["id"]  # Obtener el ID del artista
        genre = get_artist_genre(artist_id, access_token)  # Obtener el género del artista

        # Obtener características acústicas y musicales de la canción
        audio_features = get_audio_features(track_id, access_token)
        if audio_features:
            valence = audio_features.get("valence")
            acousticness = audio_features.get("acousticness")
            liveness = audio_features.get("liveness")
            danceability = audio_features.get("danceability")
            energy = audio_features.get("energy")
            instrumentalness = audio_features.get("instrumentalness")
            speechiness = audio_features.get("speechiness")
        else:
            valence = acousticness = liveness = None
            danceability = energy = instrumentalness = speechiness = None

        return {
            "name": name,
            "artist": artist,
            "genre": genre,
            "valence": valence,
            "acousticness": acousticness,
            "liveness": liveness,
            "danceability": danceability,
            "energy": energy,
            "instrumentalness": instrumentalness,
            "speechiness": speechiness
        }
    else:
        print(f"Error al obtener detalles de la canción {track_id}: {response.json()}")
        return None


1. **search_playlist(query, access_token)**:
Busca una playlist en Spotify a partir de un nombre (query) y devuelve el ID de la primera playlist que coincida con ese nombre.
Si no se encuentra ninguna playlist, devuelve None.

2. **get_track_ids_from_playlist(playlist_id, access_token)**: Obtiene los IDs de todas las canciones en una playlist a partir de su ID.

3. **get_artist_genre(artist_id, access_token)**: Obtiene el género principal de un artista a partir de su ID.
Realiza una solicitud a la API de Spotify para obtener los detalles de un artista. Extrae y devuelve el primer género listado para ese artista, o "Unknown" si no hay géneros disponibles.

4. **get_audio_features(track_id, access_token)**:
Obtiene las características acústicas de una canción a partir de su ID (como valence, acousticness, liveness, etc.).

5. **get_track_details(track_id, access_token)**:Obtiene detalles completos de una canción, incluyendo el nombre, artista, género y características acústicas.


###Playlists

In [ ]:
playlist_name = "Top 50 Mundo"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: El Top 50 de Global (ID: 37i9dQZEVXbMDoHDwVN2tF)

IDs de canciones en la playlist seleccionada:
['5vNRhkKd0yEAg8suGBpjeY', '2plbrEY59IikOBgBGLjaoe', '0nj9Bq5sHDiTxSHunhgkFb', '2CGNAOSuO1MEFCbBRgUzjd', '5gOfC9UzZQzTyShqPMrpjT', '0aB0v4027ukVziUGwVGYpG', '6dOtVTDdiauQNBQEDOtlAB', '5S8VwnB4sLi6W0lYTWYylu', '5ho7VSXSmI2KM2nDjcnLyz', '0RgjEkSbeuStKfT2Pa4Zai', '7ne4VBA60CxGM75vw0EYad', '7tI8dRuH2Yc6RuoTjxo4dU', '4K1Pg0FLno1ltzX3jeqT83', '1SGvjfc85yzqKXsfKcCxn2', '2Uts1QFB4u2YNIMiqcb4de', '0bYg9bo50gSsH3LtXe2SQn', '50zAEIE4B1QqhPjRMK2Xmh', '1cOboCuWYI2osTOfolMRS6', '2262bWmqomIaJXwCRHr13j', '06qMRF18gwbOYYbnP2du6i', '2HRqTpkrJO5ggZyyK6NPWz', '3aZptNYC6Z1YoumeqZcDcQ', '0wgOhYnqZKjOHr6bmdz0aN', '5G2f63n7IPVPPjfNIGih7Q', '3QaPy1KgI7nu9FJEQUgn6h', '0l0vcZMU7AOeQmUIREoI2U', '6WatFBLVB0x077xWeoVc2k', '1Es7AUAhQvapIcoh3qMKDL', '0WbMK4wrZ1wFSty9F7FCgu', '3xkHsmpQCBMytMJNiDf3Ii', '2EjXfH91m7f8HiJN1yQg97', '1YYfyJ2yCvGFCxwjsIqEK9', '2tHwzyyOLoWSFqYNjeVMzj', '7vQbuQcyTflfCIOu3Uzzya'

In [ ]:
track_ids = ['5vNRhkKd0yEAg8suGBpjeY', '2plbrEY59IikOBgBGLjaoe', '0nj9Bq5sHDiTxSHunhgkFb', '2CGNAOSuO1MEFCbBRgUzjd', '5gOfC9UzZQzTyShqPMrpjT', '0aB0v4027ukVziUGwVGYpG', '6dOtVTDdiauQNBQEDOtlAB', '5S8VwnB4sLi6W0lYTWYylu', '5ho7VSXSmI2KM2nDjcnLyz', '0RgjEkSbeuStKfT2Pa4Zai', '7ne4VBA60CxGM75vw0EYad', '7tI8dRuH2Yc6RuoTjxo4dU', '4K1Pg0FLno1ltzX3jeqT83', '1SGvjfc85yzqKXsfKcCxn2', '2Uts1QFB4u2YNIMiqcb4de', '0bYg9bo50gSsH3LtXe2SQn', '50zAEIE4B1QqhPjRMK2Xmh', '1cOboCuWYI2osTOfolMRS6', '2262bWmqomIaJXwCRHr13j', '06qMRF18gwbOYYbnP2du6i', '2HRqTpkrJO5ggZyyK6NPWz', '3aZptNYC6Z1YoumeqZcDcQ', '0wgOhYnqZKjOHr6bmdz0aN', '5G2f63n7IPVPPjfNIGih7Q', '3QaPy1KgI7nu9FJEQUgn6h', '0l0vcZMU7AOeQmUIREoI2U', '6WatFBLVB0x077xWeoVc2k', '1Es7AUAhQvapIcoh3qMKDL', '0WbMK4wrZ1wFSty9F7FCgu', '3xkHsmpQCBMytMJNiDf3Ii', '2EjXfH91m7f8HiJN1yQg97', '1YYfyJ2yCvGFCxwjsIqEK9', '2tHwzyyOLoWSFqYNjeVMzj', '7vQbuQcyTflfCIOu3Uzzya', '17phhZDn6oGtzMe56NuWvj', '0KKkJNfGyhkQ5aFogxQAPU', '2QjOHCTQ1Jl3zawyYOpxh6', '0lizgQ7Qw35od7CYaoMBZb', '3hRV0jL3vUpRrcy398teAU', '7KA4W4McWYRpgf0fWsJZWB', '51rfRCiUSvxXlCSCfIztBy', '5XeFesFbtLpXzIVDNQP22n', '02CrqOYzrJR8fYOffhvRZZ', '3w3y8KPTfNeOKPiqUTakBh', '1EDPVGbyPKJPeGqATwXZvN', '3tFed7YsjGnIfxeLEQwx3R', '7uoFMmxln0GPXQ0AcCBXRq', '5fZJQrFKWQLb7FpJXZ1g7K', '2uFaJJtFpPDc5Pa95XzTvg', '3AJwUDP919kvQ9QcozQPxg']
for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']} ")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.1)

Nombre de la canción: APT.
Artista: ROSÉ
Género: k-pop
Valence: 0.939
Acousticness: 0.0283
Liveness: 0.355
Danceability: 0.777 
Energy: 0.783 
Instrumentalness: 0 
Speechiness: 0.26 
---
Nombre de la canción: Die With A Smile
Artista: Lady Gaga
Género: art pop
Valence: 0.535
Acousticness: 0.308
Liveness: 0.122
Danceability: 0.521 
Energy: 0.592 
Instrumentalness: 0 
Speechiness: 0.0304 
---
Nombre de la canción: squabble up
Artista: Kendrick Lamar
Género: conscious hip hop
Valence: 0.711
Acousticness: 0.0206
Liveness: 0.0783
Danceability: 0.874 
Energy: 0.672 
Instrumentalness: 0 
Speechiness: 0.198 
---
Nombre de la canción: luther (with sza)
Artista: Kendrick Lamar
Género: conscious hip hop
Valence: 0.576
Acousticness: 0.251
Liveness: 0.248
Danceability: 0.707 
Energy: 0.575 
Instrumentalness: 0 
Speechiness: 0.125 
---
Nombre de la canción: wacced out murals
Artista: Kendrick Lamar
Género: conscious hip hop
Valence: 0.502
Acousticness: 0.249
Liveness: 0.136
Danceability: 0.761 
Ener

In [ ]:
playlist_name = "Musicoterapia Team"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)
    track_ids = track_ids[:50]

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: Música Reiki - Música de sanación emocional y física (ID: 2YgSVgtMmk0tPf3gLFDvUk)

IDs de canciones en la playlist seleccionada:
['6olL35oxaHKwAIbCq7ZKkI', '7KZmS7RwZre3XSo4djLeCm', '3WOEItuyeVks887d0MugRh', '5E1DFQFAYnWyUBqZax3xHQ', '5l3DkOMIoXwsaBq1es9yPw', '2gys1NFOM2EILZ1ONfNiOT', '5hx86Ej3OuwI4xotoWLKdC', '0sYPi9AU0HjxIOjrNVrQ7Z', '5g9hIV4oJkJt2miaCXPqE2', '33RU6DPc5e9EysSCP1wiOK', '4jwrfGZNGbINjJbh9XY50I', '7tJcdqVw0uoc6BTCLWIbHJ', '2QbVYb5f7tHOlxuQvGnu4Z', '2R56diTZRg6puXNlu4RCzX', '2MdNXLa2GKFD6ouK1EMUBF', '0ropVplLel0rHqmL8PSdBp', '6r8Ogzxx3R6ZiLUFkp3a5v', '3NdwTCoaXzrIVqW018WNKp', '1BVneOmiTPdDZZqe5SfnLM', '5Cd693PZCivuvc8tFS1j2w', '4sCct0bWGvxdpsGCBSvhTc', '4TB6X09euD0qMSX7t18v8S', '19LGXrbqLwcEt8cOdTxl9V', '3jLt5i2ChS9sQT5ZOOAz65', '3XWkCXFG7iAB6aDtsuftrG', '3AWvF5s2MpaCkuGjBt8dCy', '6w0v02g9Km85QCNoEA8YZq', '09lN8PGehiXUhF9g6uYvNu', '6v5PT0bmDRaJMpRxJTb804', '6mDVGG1YdTBveBhL8HEnd0', '5lhuIDBsQ7OfQAGzbRpdcL', '5LX3PvVATanuJ79TL3Mdc0', '2FjY8OG9VfoYwnMV

In [ ]:
track_ids =['6olL35oxaHKwAIbCq7ZKkI', '7KZmS7RwZre3XSo4djLeCm', '3WOEItuyeVks887d0MugRh', '5E1DFQFAYnWyUBqZax3xHQ', '5l3DkOMIoXwsaBq1es9yPw', '2gys1NFOM2EILZ1ONfNiOT', '5hx86Ej3OuwI4xotoWLKdC', '0sYPi9AU0HjxIOjrNVrQ7Z', '5g9hIV4oJkJt2miaCXPqE2', '33RU6DPc5e9EysSCP1wiOK', '4jwrfGZNGbINjJbh9XY50I', '7tJcdqVw0uoc6BTCLWIbHJ', '2QbVYb5f7tHOlxuQvGnu4Z', '2R56diTZRg6puXNlu4RCzX', '2MdNXLa2GKFD6ouK1EMUBF', '0ropVplLel0rHqmL8PSdBp', '6r8Ogzxx3R6ZiLUFkp3a5v', '3NdwTCoaXzrIVqW018WNKp', '1BVneOmiTPdDZZqe5SfnLM', '5Cd693PZCivuvc8tFS1j2w', '4sCct0bWGvxdpsGCBSvhTc', '4TB6X09euD0qMSX7t18v8S', '19LGXrbqLwcEt8cOdTxl9V', '3jLt5i2ChS9sQT5ZOOAz65', '3XWkCXFG7iAB6aDtsuftrG', '3AWvF5s2MpaCkuGjBt8dCy', '6w0v02g9Km85QCNoEA8YZq', '09lN8PGehiXUhF9g6uYvNu', '6v5PT0bmDRaJMpRxJTb804', '6mDVGG1YdTBveBhL8HEnd0', '5lhuIDBsQ7OfQAGzbRpdcL', '5LX3PvVATanuJ79TL3Mdc0', '2FjY8OG9VfoYwnMV3Y7zsY', '7mMxuEC8o3jtAVvdDJaz39', '6hHMthHlk0tArE1hl7jnSM', '7e22KtvJSUw9eyQf1G88Y2', '22uoTUrLBebr097n9iwSpL', '7IuZXkt3GOLwWf7GBUZAwP', '7s44sqfDnRkVc0OS6I5Lrz', '4uYaZMa7dLll3hd71Oj8Pv', '3EyUDtd86opZMse6grZAFo', '77iqH1BetYkfENOYUZ93fc', '4ef4JczpiSaYlmtMNboRMi', '1IR6eynHRxpDbjji1jybW0', '44oYGNdNjIf3BogQrOuRp4', '7p3l2yCTrZhuUEzIY3pplp', '5iKRdjGq1888F8ZMdKeZud', '49mBEZJ3kpYjEMqAJisHRu', '1iJBAQBhZhx9WKESQDz4y5', '2MqKBGEPehas7MpdZfKEUN']
for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']} ")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.1)

Nombre de la canción: Reiki Harmony
Artista: Spiritual Moment
Género: tone
Valence: 0.0395
Acousticness: 0.817
Liveness: 0.147
Danceability: 0.134 
Energy: 0.00834 
Instrumentalness: 0.961 
Speechiness: 0.0453 
---
Nombre de la canción: Reiki Healing
Artista: Spiritual Moment
Género: tone
Valence: 0.04
Acousticness: 0.746
Liveness: 0.102
Danceability: 0.173 
Energy: 0.00936 
Instrumentalness: 0.918 
Speechiness: 0.0553 
---
Nombre de la canción: Spes
Artista: kailon
Género: Unknown
Valence: 0.0368
Acousticness: 0.743
Liveness: 0.112
Danceability: 0.145 
Energy: 0.0563 
Instrumentalness: 0.639 
Speechiness: 0.0384 
---
Nombre de la canción: Chakra Flow
Artista: Spiritual Moment
Género: tone
Valence: 0.04
Acousticness: 0.959
Liveness: 0.0829
Danceability: 0.118 
Energy: 0.00712 
Instrumentalness: 0.982 
Speechiness: 0.0475 
---
Nombre de la canción: Reiki Energy
Artista: Spiritual Moment
Género: tone
Valence: 0.0378
Acousticness: 0.832
Liveness: 0.102
Danceability: 0.0732 
Energy: 0.0087

In [ ]:
playlist_name = "Rock"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)
    track_ids = track_ids[:50]

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: Rock Nacional: 100% Clásicos (ID: 37i9dQZF1DWSo7PX7dbgH8)

IDs de canciones en la playlist seleccionada:
['67Q93kXygxCskeQiIP0V0j', '6Pur3hWy6Nzc27ilmsp5HA', '3yJKzKUDLo65qDDBDZm9im', '1p7m9H4H8s0Y7SgRm7j3ED', '0oCiPqIEL5218yu2D996eP', '65DBZofI0b79kfHTcWWDuU', '2tWlLCwEN7slVfcPjemDYJ', '0CgpCtEPvyDizcg2p47VrU', '5QpdTr2fHXLF0Oj8TcGHmV', '198qSChSMQFuSimdeeY9gK', '6Soxxi73U6SO4rOGJwYYxI', '3oqWr0jDWNXxWufNogGREp', '5vHWh24GAsXJ7tnIu8v7UN', '4eCrF1FgVL4se3AkHOnLDL', '71awpJoi5bqGMBrTkHDDoL', '3ydweVPwq9PCEVJ8WwqT6u', '6MuJUSYCqA4FpMogUHzb2R', '61qPUnazSdkvua4wgA4L8C', '1rh4kDY9T4fHVDum8Foi5k', '1lAFWiaVhJhvQ5Gtzp6vSu', '2PkTjm1QAJCZbJ9MqC4RqA', '4k3K6Ke6KfAWb7zyQWrTEA', '44m9PAUZYeOEGeaTYh6NAl', '1orgfjGJoycODSItcGVatE', '6gwaa6ElIixNTvu6RwkMyo', '0fHTVPRRKmWaRuIwrsNTMp', '2qBirMakpTdz9ymxrZEyzg', '0ZAJ660VP57lLK4U7NlGOy', '0dsViRiDTIuexAL42Nc1Kh', '14rcLGX83RpsORAIzC4U4g', '2RognU2ViRdA6HxnpAITJl', '4VikOud5ZmdmHH6h7uQeDB', '7upAH0ezzsThyvr33YKn4u', '2jEjxOyKqMpkcT

In [ ]:
track_id = ['67Q93kXygxCskeQiIP0V0j', '6Pur3hWy6Nzc27ilmsp5HA', '3yJKzKUDLo65qDDBDZm9im', '1p7m9H4H8s0Y7SgRm7j3ED', '0oCiPqIEL5218yu2D996eP', '65DBZofI0b79kfHTcWWDuU', '2tWlLCwEN7slVfcPjemDYJ', '0CgpCtEPvyDizcg2p47VrU', '5QpdTr2fHXLF0Oj8TcGHmV', '198qSChSMQFuSimdeeY9gK', '6Soxxi73U6SO4rOGJwYYxI', '3oqWr0jDWNXxWufNogGREp', '5vHWh24GAsXJ7tnIu8v7UN', '4eCrF1FgVL4se3AkHOnLDL', '71awpJoi5bqGMBrTkHDDoL', '3ydweVPwq9PCEVJ8WwqT6u', '6MuJUSYCqA4FpMogUHzb2R', '61qPUnazSdkvua4wgA4L8C', '1rh4kDY9T4fHVDum8Foi5k', '1lAFWiaVhJhvQ5Gtzp6vSu', '2PkTjm1QAJCZbJ9MqC4RqA', '4k3K6Ke6KfAWb7zyQWrTEA', '44m9PAUZYeOEGeaTYh6NAl', '1orgfjGJoycODSItcGVatE', '6gwaa6ElIixNTvu6RwkMyo', '0fHTVPRRKmWaRuIwrsNTMp', '2qBirMakpTdz9ymxrZEyzg', '0ZAJ660VP57lLK4U7NlGOy', '0dsViRiDTIuexAL42Nc1Kh', '14rcLGX83RpsORAIzC4U4g', '2RognU2ViRdA6HxnpAITJl', '4VikOud5ZmdmHH6h7uQeDB', '7upAH0ezzsThyvr33YKn4u', '2jEjxOyKqMpkcThWl697hf', '6WgUNHiMwJazeG4kWPAH1m', '6iCnJkKswtMYoSmT58HmEn', '7qIw4D1PcDHigDohKkLB0W', '5Bi9Cq5oVFy1OtUgecbegZ', '6kUopck7LHVadAjAFcXiTj', '1PQzZbitOJ6XPFg7FFzsKQ', '2pmHBQiIjVfTyn3p1rRbSB', '29iIRrSuANyjnwag69PHOJ', '0N361WhEqVunVl5ZBkfd70', '0SaKhiBPgdRaQKA7Bxu3ek', '56mbNpJVn5WQhOlsOvb2A0', '1Ev18ptTlc3CBQMmKsIfjY', '2zogXcCqvKJwh3o67W3GG3', '04bg4edMdpKhsgkSzYqj55', '2kB1rqoTaZ3j6ChtIWGurm', '0C9V5jMElp7laZZdsCHufD']

for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']} ")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.1)

Nombre de la canción: Me Gusta
Artista: Ciro y los Persas
Género: argentine rock
Valence: 0.706
Acousticness: 0.0204
Liveness: 0.131
Danceability: 0.643 
Energy: 0.813 
Instrumentalness: 5.48e-05 
Speechiness: 0.0272 
---
Nombre de la canción: Lamento Boliviano
Artista: Los Enanitos Verdes
Género: argentine rock
Valence: 0.68
Acousticness: 0.176
Liveness: 0.115
Danceability: 0.714 
Energy: 0.735 
Instrumentalness: 0.000622 
Speechiness: 0.0306 
---
Nombre de la canción: Como Alí
Artista: Los Piojos
Género: argentine rock
Valence: 0.928
Acousticness: 0.00228
Liveness: 0.118
Danceability: 0.63 
Energy: 0.949 
Instrumentalness: 0 
Speechiness: 0.0334 
---
Nombre de la canción: Flaca
Artista: Andrés Calamaro
Género: argentine rock
Valence: 0.949
Acousticness: 0.21
Liveness: 0.127
Danceability: 0.739 
Energy: 0.745 
Instrumentalness: 0.0135 
Speechiness: 0.0267 
---
Nombre de la canción: Bicho de Ciudad
Artista: Los Piojos
Género: argentine rock
Valence: 0.395
Acousticness: 0.296
Liveness: 

In [ ]:
playlist_name = "lounge"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)
    track_ids = track_ids[:50]

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: Lounge Music 2024 🍹 - Soft House - Ibiza Lounge - Chill House (ID: 3ZrIRwh11hu5L9XC42ohmm)

IDs de canciones en la playlist seleccionada:
['3uUuGVFu1V7jTQL60S1r8z', '1BJJbSX6muJVF2AK7uH1x4', '0b02ZHphmQGm0lRtHyDDTH', '1LY3GhF0zxIVgbYEQjCbUO', '4YM1Jcr8dPy1Ql0khFMt4L', '5nPbKG04fhLkIAjcPFaZq7', '5EXfgxpWz0mGoq9hjwbCMm', '5KTZgG84bKFGm53lhLtTqc', '0e42FrN75U6RuIzBTqszKz', '5LOTtadWBsGDLMQMC0tl2z', '55QiyxSKStXrVhS4vUA6jA', '1yoO94wPL8khknS04ltxkn', '6tZVfJMk9iuLNLmagoensg', '1AhJvwSNM0VcQClYZkpyZd', '4255amV4enzl28KAn16rUO', '05rCEPSYajDS8Y57HAKPxf', '66bnX8ozY4o8FATgrQ7Bd0', '1xfj8i1OvEhqJmWncOV2d5', '1HFfMOxCAT4GAwaPfCdmUs', '5ztq1zM2wUPgFqr2PbwXj6', '3yNHaAkNBDIZ10HxcWsuvC', '4iJQwEWqYRMWXXlk0AzyEv', '0h3Xy4V4apMraB5NuM8U7Z', '6sDP6lOELyA6maxV32Ene9', '6z3rjm8KusS1JnDCOIMbHK', '23RoR84KodL5HWvUTneQ1w', '5wZQ4MV8e5xPKPcwIXAOLL', '6MWqosLyBiQi4myJ86PnVj', '05GvwwTLLID738BbKN1ze0', '4WDV17qUIDMTrXTX5hCuzu', '1aRg9ACpBToG4ess6tczVe', '4MSj19TwYBLgDFj3ddEeco', '6Y3bcR1

In [ ]:
track_id =['3uUuGVFu1V7jTQL60S1r8z', '1BJJbSX6muJVF2AK7uH1x4', '0b02ZHphmQGm0lRtHyDDTH', '1LY3GhF0zxIVgbYEQjCbUO', '4YM1Jcr8dPy1Ql0khFMt4L', '5nPbKG04fhLkIAjcPFaZq7', '5EXfgxpWz0mGoq9hjwbCMm', '5KTZgG84bKFGm53lhLtTqc', '0e42FrN75U6RuIzBTqszKz', '5LOTtadWBsGDLMQMC0tl2z', '55QiyxSKStXrVhS4vUA6jA', '1yoO94wPL8khknS04ltxkn', '6tZVfJMk9iuLNLmagoensg', '1AhJvwSNM0VcQClYZkpyZd', '4255amV4enzl28KAn16rUO', '05rCEPSYajDS8Y57HAKPxf', '66bnX8ozY4o8FATgrQ7Bd0', '1xfj8i1OvEhqJmWncOV2d5', '1HFfMOxCAT4GAwaPfCdmUs', '5ztq1zM2wUPgFqr2PbwXj6', '3yNHaAkNBDIZ10HxcWsuvC', '4iJQwEWqYRMWXXlk0AzyEv', '0h3Xy4V4apMraB5NuM8U7Z', '6sDP6lOELyA6maxV32Ene9', '6z3rjm8KusS1JnDCOIMbHK', '23RoR84KodL5HWvUTneQ1w', '5wZQ4MV8e5xPKPcwIXAOLL', '6MWqosLyBiQi4myJ86PnVj', '05GvwwTLLID738BbKN1ze0', '4WDV17qUIDMTrXTX5hCuzu', '1aRg9ACpBToG4ess6tczVe', '4MSj19TwYBLgDFj3ddEeco', '6Y3bcR1FxXAABHNjxxjArB', '0pPe0raqvqitErFIsbcuTF', '2tnVG71enUj33Ic2nFN6kZ', '7hKfmYaKcfqr9zBkVYaexC', '2sEz5OaGg1yI3FRHE2smi7', '64EBmfpcf3bj3A8rPoVadm', '1t0Jmqg1pKVBbxjQFZebeR', '3EP9qywZJ8Mapq5akA3FFx', '1gPzf8Se4hV8Ex0ylGcTbp', '61UQzeiIluhpzpMdY4ag3q', '0XIICwmd3FAoU6vpbdQfcn', '2zft26Rrg30VUTSyxxmQ8C', '5masKPHeAOVNgxdLebIcK7', '1vypt0hOMOY82Vr8ybWlzM', '4pvoOCoh4uhBGcMEdHQ3kP', '5OFVzqSeFxGpvDGyHvVeLj', '3NuM9FD8IWiRaEmMbkz1HV', '0s4xhRPu3mksDPQdvs2zy2']
for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']} ")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.1)

Nombre de la canción: Where Are You Now
Artista: Lost Frequencies
Género: belgian edm
Valence: 0.262
Acousticness: 0.515
Liveness: 0.172
Danceability: 0.671 
Energy: 0.636 
Instrumentalness: 0.000411 
Speechiness: 0.103 
---
Nombre de la canción: Move
Artista: Adam Port
Género: melodic house
Valence: 0.655
Acousticness: 0.0563
Liveness: 0.225
Danceability: 0.727 
Energy: 0.782 
Instrumentalness: 0.336 
Speechiness: 0.042 
---
Nombre de la canción: Run Away
Artista: Ian Storm
Género: dutch edm
Valence: 0.397
Acousticness: 0.292
Liveness: 0.141
Danceability: 0.593 
Energy: 0.852 
Instrumentalness: 0.000162 
Speechiness: 0.119 
---
Nombre de la canción: The Sound of Silence - CYRIL Remix
Artista: Disturbed
Género: alternative metal
Valence: 0.244
Acousticness: 0.123
Liveness: 0.162
Danceability: 0.68 
Energy: 0.721 
Instrumentalness: 0 
Speechiness: 0.188 
---
Nombre de la canción: Losing My Religion
Artista: Ian Storm
Género: dutch edm
Valence: 0.331
Acousticness: 0.0617
Liveness: 0.393


In [ ]:
playlist_name = "2000s pop"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)
    track_ids = track_ids[:50]

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: 2000s Pop Hits (ID: 3p7nFyrNADyD3sduglPDZN)

IDs de canciones en la playlist seleccionada:
['6I9VzXrHxO9rA9A5euc8Ak', '62bOmKYxYg7dhrC6gH9vFn', '1KGi9sZVMeszgZOWivFpxs', '47BBI51FKFwOMlIiX6m8ya', '7H6ev70Weq6DdpZyyTmUXk', '6xkryXuiZU360Lngd4sx13', '2VjXGuPVVxyhMgER3Uz2Fe', '1Hv1VTm8zeOeybub15mA2R', '3XKIUb7HzIF1Vu9usunMzc', '7A0apkTSTvMbSI7yplcmlh', '1Je1IMUlBXcx1Fz0WE7oPT', '503OTo2dSqe7qk76rgsbep', '3MjUtNVVq3C8Fn0MP3zhXa', '6qspW4YKycviDFjHBOaqUY', '51UtgWS4z1eMPuLQOzPtNH', '2AW37v0bDyuOzGP3XnmFuA', '0weAUscowxeqDtpCgtbpgp', '3x4yV0hW5Ve3TKhFkXSqFn', '1DIXPcTDzTj8ZMHt3PDt8p', '2aBxt229cbLDOvtL7Xbb9x', '4pmc2AxSEq6g7hPVlJCPyP', '594M0rqYMOo8BhMGEdoi5C', '6kvoHl80mfCVTv7XnZkjQn', '6x4tKaOzfNJpEJHySoiJcs', '0n2SEXB2qoRQg171q7XqeW', '5yaCquc7koPqtgj7v0lwHX', '57iDDD9N9tTWe75x6qhStw', '1aWV3uY3SIEZVbmv45oFWS', '6naxalmIoLFWR0siv8dnQQ', '7EKvTUo567BRNzZ37clvlf', '0Ph6L4l8dYUuXFmb71Ajnd', '1G391cbiT3v3Cywg8T7DM1', '6e8Ou0wiqAzIpWb2eSxll8', '5TAf4lnZCZTLlZHNZMLFLi', '2C

In [ ]:
track_id = ['6I9VzXrHxO9rA9A5euc8Ak', '62bOmKYxYg7dhrC6gH9vFn', '1KGi9sZVMeszgZOWivFpxs', '47BBI51FKFwOMlIiX6m8ya', '7H6ev70Weq6DdpZyyTmUXk', '6xkryXuiZU360Lngd4sx13', '2VjXGuPVVxyhMgER3Uz2Fe', '1Hv1VTm8zeOeybub15mA2R', '3XKIUb7HzIF1Vu9usunMzc', '7A0apkTSTvMbSI7yplcmlh', '1Je1IMUlBXcx1Fz0WE7oPT', '503OTo2dSqe7qk76rgsbep', '3MjUtNVVq3C8Fn0MP3zhXa', '6qspW4YKycviDFjHBOaqUY', '51UtgWS4z1eMPuLQOzPtNH', '2AW37v0bDyuOzGP3XnmFuA', '0weAUscowxeqDtpCgtbpgp', '3x4yV0hW5Ve3TKhFkXSqFn', '1DIXPcTDzTj8ZMHt3PDt8p', '2aBxt229cbLDOvtL7Xbb9x', '4pmc2AxSEq6g7hPVlJCPyP', '594M0rqYMOo8BhMGEdoi5C', '6kvoHl80mfCVTv7XnZkjQn', '6x4tKaOzfNJpEJHySoiJcs', '0n2SEXB2qoRQg171q7XqeW', '5yaCquc7koPqtgj7v0lwHX', '57iDDD9N9tTWe75x6qhStw', '1aWV3uY3SIEZVbmv45oFWS', '6naxalmIoLFWR0siv8dnQQ', '7EKvTUo567BRNzZ37clvlf', '0Ph6L4l8dYUuXFmb71Ajnd', '1G391cbiT3v3Cywg8T7DM1', '6e8Ou0wiqAzIpWb2eSxll8', '5TAf4lnZCZTLlZHNZMLFLi', '2Cy7QY8HPLk925AyNAt6OG', '1DSJNBNhGZCigg9ll5VeZv', '31bf9SEOppLU6lQ85d8om6', '4LJhJ6DQS7NwE7UKtvcM52', '3WkibOpDF7cQ5xntM1epyf', '2FK7fxjzQEXD7Z32HSF0Hl', '3Ti0GdlrotgwsAVBBugv0I', '33LC84JgLvK2KuW43MfaNq', '3cfOd4CMv2snFaKAnMdnvK', '0aPaGIX2QIMAH5SMp0VEMe', '3vZO25GdYuqFrR1kzZADnp', '013AWvizllIUEC2FOBzOnh', '754kgU5rWscRTfvlsuEwFp', '1Jaah2tmN9Hv81A87KZ1MU', '11mwFrKvLXCbcVGNxffGyP', '0lnxrQAd9ZxbhBBe7d8FO8']
for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']} ")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.1)

Nombre de la canción: Toxic
Artista: Britney Spears
Género: dance pop
Valence: 0.924
Acousticness: 0.0249
Liveness: 0.242
Danceability: 0.774 
Energy: 0.838 
Instrumentalness: 0.025 
Speechiness: 0.114 
---
Nombre de la canción: Bye Bye Bye - From Deadpool and Wolverine Soundtrack
Artista: *NSYNC
Género: boy band
Valence: 0.861
Acousticness: 0.031
Liveness: 0.0821
Danceability: 0.61 
Energy: 0.926 
Instrumentalness: 0.0012 
Speechiness: 0.0479 
---
Nombre de la canción: No Scrubs
Artista: TLC
Género: atl hip hop
Valence: 0.59
Acousticness: 0.0251
Liveness: 0.0685
Danceability: 0.743 
Energy: 0.675 
Instrumentalness: 0.000717 
Speechiness: 0.0953 
---
Nombre de la canción: I Want It That Way
Artista: Backstreet Boys
Género: boy band
Valence: 0.482
Acousticness: 0.257
Liveness: 0.148
Danceability: 0.689 
Energy: 0.694 
Instrumentalness: 0 
Speechiness: 0.027 
---
Nombre de la canción: Say My Name
Artista: Destiny's Child
Género: dance pop
Valence: 0.734
Acousticness: 0.273
Liveness: 0.14

In [ ]:
playlist_name = "Classic"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)
    track_ids = track_ids[:50]

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: Classical music for villains  (ID: 0PLteLOoJNWvHh7OdxugTJ)

IDs de canciones en la playlist seleccionada:
['2cTJZdJuudcg4JB66koO5K', '5OC5sgs7B6ieroHZmcjO1o', '5Zf25eS8E1znm9mez4cGsm', '2xizRhme7pYeITbH1NLLGt', '3DNRdudZ2SstnDCVKFdXxG', '0ROMalDdNg3L2HoZP4qFyJ', '2CsjpNrIV7BpkgSW2GncpS', '3A8br9jzxNJPLeyEzqmlfg', '5kqLPvBEL6NXrs2xxJFy4K', '1YTXSyOSmL4ijaesMND04A', '1npqP0Epwvvc1jzudkkmVB', '1EjfCGPicAauZwnbFrTczK', '519SD3ON58hqmrAMALWiWp', '5nnnoeboXoY57eZPDsPr4i', '4wgS4pen0Kej9PafGhUhMJ', '2MSgFefjK0T7Iwjvr3OKqV', '2AjyreHCD6PKAtRGV2qsK5', '1Xf4sNtMKc1xiUhuIPAUg2', '7xfSCgVOkQJhVxnqzepATH', '0SKn1ZAGun3ekX1LzfKgMz', '4Yj5BlxJnOm6wn5DERt1H6', '6BdrHOEmokkSxTyxCojD6r', '0V5cvmTKsYmF5FmGGEAfmS', '62P3CMQAknHW4iftsBD41Y', '5u5aVJKjSMJr4zesMPz7bL', '3zWfjRtvm3gFEAjqeWScEl', '0Knyi7Zqpbl0QsIg8nMrGV', '0bSHwuTOZVJUXWT03H9oD2', '4820SC1xnUaabzzYQTDkP2', '1cd7Pft3rfZ0hmaOLI0EDB', '1IM8x4lxZVOOP9gpQD6c5s', '7faWy3kAy0GkTqAzkUBhCf', '79RYNk7I1YDy4RLPamenAn', '0FjnJPF8c6dDD

In [ ]:
track_id = ['2cTJZdJuudcg4JB66koO5K', '5OC5sgs7B6ieroHZmcjO1o', '5Zf25eS8E1znm9mez4cGsm', '2xizRhme7pYeITbH1NLLGt', '3DNRdudZ2SstnDCVKFdXxG', '0ROMalDdNg3L2HoZP4qFyJ', '2CsjpNrIV7BpkgSW2GncpS', '3A8br9jzxNJPLeyEzqmlfg', '5kqLPvBEL6NXrs2xxJFy4K', '1YTXSyOSmL4ijaesMND04A', '1npqP0Epwvvc1jzudkkmVB', '1EjfCGPicAauZwnbFrTczK', '519SD3ON58hqmrAMALWiWp', '5nnnoeboXoY57eZPDsPr4i', '4wgS4pen0Kej9PafGhUhMJ', '2MSgFefjK0T7Iwjvr3OKqV', '2AjyreHCD6PKAtRGV2qsK5', '1Xf4sNtMKc1xiUhuIPAUg2', '7xfSCgVOkQJhVxnqzepATH', '0SKn1ZAGun3ekX1LzfKgMz', '4Yj5BlxJnOm6wn5DERt1H6', '6BdrHOEmokkSxTyxCojD6r', '0V5cvmTKsYmF5FmGGEAfmS', '62P3CMQAknHW4iftsBD41Y', '5u5aVJKjSMJr4zesMPz7bL', '3zWfjRtvm3gFEAjqeWScEl', '0Knyi7Zqpbl0QsIg8nMrGV', '0bSHwuTOZVJUXWT03H9oD2', '4820SC1xnUaabzzYQTDkP2', '1cd7Pft3rfZ0hmaOLI0EDB', '1IM8x4lxZVOOP9gpQD6c5s', '7faWy3kAy0GkTqAzkUBhCf', '79RYNk7I1YDy4RLPamenAn', '0FjnJPF8c6dDDHqmBWm48p', '630zE7N1xPxdK2Zz4S3rPA', '5RAQbPEWgR6QtPbypV5XAp', '1BncfTJAWxrsxyT9culBrj']
for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']} ")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.1)

Nombre de la canción: Song Of Time
Artista: Abel Korzeniowski
Género: compositional ambient
Valence: 0.163
Acousticness: 0.832
Liveness: 0.11
Danceability: 0.253 
Energy: 0.0434 
Instrumentalness: 0.965 
Speechiness: 0.0395 
---
Nombre de la canción: Dining With Lectre
Artista: Peter Gundry
Género: rune folk
Valence: 0.0755
Acousticness: 0.784
Liveness: 0.0687
Danceability: 0.184 
Energy: 0.204 
Instrumentalness: 0.734 
Speechiness: 0.0427 
---
Nombre de la canción: Reflections
Artista: Toshifumi Hinata
Género: anime score
Valence: 0.142
Acousticness: 0.961
Liveness: 0.113
Danceability: 0.224 
Energy: 0.146 
Instrumentalness: 0.712 
Speechiness: 0.0338 
---
Nombre de la canción: Tchaikovsky: Swan Lake, Op. 20, Act 2: No. 10, Scene. Moderato
Artista: Pyotr Ilyich Tchaikovsky
Género: classical
Valence: 0.071
Acousticness: 0.911
Liveness: 0.109
Danceability: 0.136 
Energy: 0.164 
Instrumentalness: 0.916 
Speechiness: 0.0364 
---
Nombre de la canción: Sonata No. 14 "Moonlight" in C-Sharp M

In [ ]:
playlist_name = "Chillout"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)
    track_ids = track_ids[:50]

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: Chillout 2024 (ID: 7ozIozDp260fjNOZy1yzRG)

IDs de canciones en la playlist seleccionada:
['3aHVQC1g6ArGNy3VLL4Pz7', '2SWyPUuh4xfYkH7EUH5qgZ', '4l7gfaL2EDvuJWEuLcNpvA', '0oflTn1bxRy8HLcB2TLxQ8', '06tCB0GoFpEAM9rEWiVrX8', '7KcR2bleMxNhpX9jUiSf4H', '7xNiLGpPvHIhrBoi7syMST', '7HtANTf5bkiyiuWf7GTEpB', '6tsgmTbVDKXPVZlNYVpylr', '0vlhtJfber57vjxDIQIwda', '1j2uOajBXtB0dV7gmaUIwG', '5H4s0E08rg6xIOl4hELDVr', '6bTf55POJ73MwwGAJDTxhV', '50QNhaax3ojdSdzR94sfd9', '4jXQ0BG5XYsYnAg2LDRebl', '2wk7mBr6JlorYMwESY7lyi', '3hEPKX4FgfYPhBjSw1Nejq', '3xc9WBrfURBKh7TfwWajxe', '2dNtChXYtvpT7SD7Bfe5KS', '611m4J9OHUKNvI1ZPJQLOv', '5T5rO4oqOLDHSqa6WePzz3', '7j5IEblPv4Q15WE667AlGb', '6YLthiS7dxeABA8sEkJfU8', '6JudCjkP3mZMUL5cf22QVu', '6gOFS1DicuJRiLSB4ZDorp', '4TkzQfVHEBmEAJz82OQWdR', '6y6y9WemSUN5DX1NIsbxNx', '3qTRQaYlWIfbGSu9qpHJGm', '1KaGHfN8QGVtkcYGolrDOj', '7w9NOqoeE3Zfv19VYMRMxF', '3XIs4PhllxzbfjsbeSIzM9', '5pB5ViaicDA1RGPcR8WsYb', '7CI2TUAm83hUc1ORjw7DKT', '5eREa3J3z7KSv0Lx2zNS1M', '4zg

In [ ]:
track_id =['3aHVQC1g6ArGNy3VLL4Pz7', '2SWyPUuh4xfYkH7EUH5qgZ', '4l7gfaL2EDvuJWEuLcNpvA', '0oflTn1bxRy8HLcB2TLxQ8', '06tCB0GoFpEAM9rEWiVrX8', '7KcR2bleMxNhpX9jUiSf4H', '7xNiLGpPvHIhrBoi7syMST', '7HtANTf5bkiyiuWf7GTEpB', '6tsgmTbVDKXPVZlNYVpylr', '0vlhtJfber57vjxDIQIwda', '1j2uOajBXtB0dV7gmaUIwG', '5H4s0E08rg6xIOl4hELDVr', '6bTf55POJ73MwwGAJDTxhV', '50QNhaax3ojdSdzR94sfd9', '4jXQ0BG5XYsYnAg2LDRebl', '2wk7mBr6JlorYMwESY7lyi', '3hEPKX4FgfYPhBjSw1Nejq', '3xc9WBrfURBKh7TfwWajxe', '2dNtChXYtvpT7SD7Bfe5KS', '611m4J9OHUKNvI1ZPJQLOv', '5T5rO4oqOLDHSqa6WePzz3', '7j5IEblPv4Q15WE667AlGb', '6YLthiS7dxeABA8sEkJfU8', '6JudCjkP3mZMUL5cf22QVu', '6gOFS1DicuJRiLSB4ZDorp', '4TkzQfVHEBmEAJz82OQWdR', '6y6y9WemSUN5DX1NIsbxNx', '3qTRQaYlWIfbGSu9qpHJGm', '1KaGHfN8QGVtkcYGolrDOj', '7w9NOqoeE3Zfv19VYMRMxF', '3XIs4PhllxzbfjsbeSIzM9', '5pB5ViaicDA1RGPcR8WsYb', '7CI2TUAm83hUc1ORjw7DKT', '5eREa3J3z7KSv0Lx2zNS1M', '4zgvEgmgyUrH9yJrvf13Il', '0R0amVg7jBzXUfvf34GVjN', '7ldEYrgs3NoGNFTFiJu5AB', '6SIgxgVNDe7ppyUqWEbqba', '0J3BeS2Q80ae13CB9I4Mue', '73RdmHN6iP6pAAogqSR8oH', '3GKzjPk3aIHhAxZkPxophD', '6K4FiCPiCn5HspmocEUBMa', '2RFh2VDNrqky21qASrZTRi', '5dRb6JTLqLSquLy4UullV8', '2Yg1eqr6bPvH77baHLQRE7', '5azsq59LqhM0Ubd0fUq5jx', '2vJo9uxQODvG8PiIpfUzFW', '0xSu8TL6l2U258VV43xEDO', '4rLVsInGwwMytBFC1hCmT1', '6L68S4OwRdPLPZCP8WXUpT']
for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']} ")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.1)

Nombre de la canción: London
Artista: RANE
Género: chill house
Valence: 0.0871
Acousticness: 0.348
Liveness: 0.111
Danceability: 0.737 
Energy: 0.651 
Instrumentalness: 0.0136 
Speechiness: 0.0362 
---
Nombre de la canción: Torn
Artista: Paris Blu
Género: chill house
Valence: 0.372
Acousticness: 0.419
Liveness: 0.108
Danceability: 0.707 
Energy: 0.382 
Instrumentalness: 0 
Speechiness: 0.251 
---
Nombre de la canción: Ocean Eyes
Artista: NOVUM
Género: chill house
Valence: 0.548
Acousticness: 0.0672
Liveness: 0.275
Danceability: 0.612 
Energy: 0.626 
Instrumentalness: 0.00112 
Speechiness: 0.042 
---
Nombre de la canción: I'll Be Alright
Artista: VELMA
Género: chill house
Valence: 0.201
Acousticness: 0.281
Liveness: 0.173
Danceability: 0.652 
Energy: 0.653 
Instrumentalness: 0.253 
Speechiness: 0.035 
---
Nombre de la canción: Travel
Artista: Mauve
Género: chill house
Valence: 0.263
Acousticness: 0.839
Liveness: 0.25
Danceability: 0.51 
Energy: 0.331 
Instrumentalness: 3.39e-05 
Speechi

In [ ]:
playlist_name = "Lo fi"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)
    track_ids = track_ids[:50]

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: Lofi Study 2024 (ID: 6zCID88oNjNv9zx6puDHKj)

IDs de canciones en la playlist seleccionada:
['4Lx7mjMqpeNAe3jb58QzPh', '78pATVtCPzSFJXHO5j0k4U', '2KbDWTdcBWAk3lSXadhemv', '6h0Ot63iDprbarV3qq6H0P', '0clvKlDlGGxBqy0Xsb384V', '6EmZHSVhBVINc3jUkLcxid', '5Jz0m7Rq3NWQ70CrdXiG0R', '1Pggim4sOrvSDftD5ZNaUN', '0iPJkxyzu2PkYLUJfmzers', '0K33PjWz3o40CxAP7yQu2h', '4PewNRTCRkjsFKIWEldaUz', '4TFnUYS0vbUHzHCT75WL80', '5Hpwb8l7NHJkiCZOPRmfIK', '3y2igQZuHEW2gpIYgPF4Ip', '2rzlS908ocTsEZS8KOddYl', '7q15GMbDPdxKFHDWzVCm37', '4lmR9fgYd7DGylRMN752xl', '5B3fI3C2MTVub81SLmBIxo', '2mpMZKy6Bo4y1DcXt9z8tN', '0Cu1ihqQvw3YDA5SRdP3jG', '4zRJTAbwdFzbS9Ja5VJXC1', '4IEvn9Zeh2wjqWt7lhFRIZ', '1ng4fAqBrzMQ3u7mXKU70r', '3m23t6VhVfTQbowV0X6K6p', '4xgbgngfiCpNgLNu0r2OUa', '4ssP7DejEIDLXaVywwC8iA', '6tIqT0g4EjFMXTMC49ELr1', '4lBaMOB3FJT1FScur67Pdg', '64kqtYWmDYYOxYosU1Z5N6', '2GLd8jprGu6UUnTmgDMlhN', '3NAnapBL52D4PfylWxPeJo', '5JQaS2WwqP1P4HFm7FTHfm', '6QsZc7VLKBwtYpggf5o6Iv', '4h3KM0HdN1xnYexB7zCIjG', '5

In [ ]:
track_id = ['4Lx7mjMqpeNAe3jb58QzPh', '78pATVtCPzSFJXHO5j0k4U', '2KbDWTdcBWAk3lSXadhemv', '6h0Ot63iDprbarV3qq6H0P', '0clvKlDlGGxBqy0Xsb384V', '6EmZHSVhBVINc3jUkLcxid', '5Jz0m7Rq3NWQ70CrdXiG0R', '1Pggim4sOrvSDftD5ZNaUN', '0iPJkxyzu2PkYLUJfmzers', '0K33PjWz3o40CxAP7yQu2h', '4PewNRTCRkjsFKIWEldaUz', '4TFnUYS0vbUHzHCT75WL80', '5Hpwb8l7NHJkiCZOPRmfIK', '3y2igQZuHEW2gpIYgPF4Ip', '2rzlS908ocTsEZS8KOddYl', '7q15GMbDPdxKFHDWzVCm37', '4lmR9fgYd7DGylRMN752xl', '5B3fI3C2MTVub81SLmBIxo', '2mpMZKy6Bo4y1DcXt9z8tN', '0Cu1ihqQvw3YDA5SRdP3jG', '4zRJTAbwdFzbS9Ja5VJXC1', '4IEvn9Zeh2wjqWt7lhFRIZ', '1ng4fAqBrzMQ3u7mXKU70r', '3m23t6VhVfTQbowV0X6K6p', '4xgbgngfiCpNgLNu0r2OUa', '4ssP7DejEIDLXaVywwC8iA', '6tIqT0g4EjFMXTMC49ELr1', '4lBaMOB3FJT1FScur67Pdg', '64kqtYWmDYYOxYosU1Z5N6', '2GLd8jprGu6UUnTmgDMlhN', '3NAnapBL52D4PfylWxPeJo', '5JQaS2WwqP1P4HFm7FTHfm', '6QsZc7VLKBwtYpggf5o6Iv', '4h3KM0HdN1xnYexB7zCIjG', '5fU2bSQzXMNIzfksE9vzcA', '4gFPt4zCY9eGG8mWgp2Q2f', '5raJ06cT4FHjR56o3dSE5y', '29adV3blIhgh46gYyF5Zie', '7JW6HbWC2LanRbQgOxdzsQ', '33VX7FynVkGs5fvoYdGmpR', '3mRV4GqhtvzJ1vdPalHaJj', '14fjRdR9iusUysHJHatVOB', '4Y7yehXb1FJlcmPA82lasd', '4JniCFU7RuaucBbS21CWoP', '5xyhG7nGl3A8pvFFCEbTdl', '0esWyCprYUKo0LbYVouX0h', '6L1AafdadMlmWrOQzJmJkU', '1RHFPBrrYP93UYSQo0865S', '0NEGRjfrQBYNY59Hkv8xH5', '6eeCSRVwpCn9VCiIRUsIhs']
for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']} ")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.1)

Nombre de la canción: clear eyes, blind sight
Artista: idylla
Género: lo-fi study
Valence: 0.063
Acousticness: 0.913
Liveness: 0.137
Danceability: 0.47 
Energy: 0.0779 
Instrumentalness: 0.959 
Speechiness: 0.0377 
---
Nombre de la canción: Chrono
Artista: Monma
Género: chillhop
Valence: 0.627
Acousticness: 0.135
Liveness: 0.13
Danceability: 0.782 
Energy: 0.159 
Instrumentalness: 0.905 
Speechiness: 0.0742 
---
Nombre de la canción: Don't Let Her Go
Artista: xander.
Género: lo-fi study
Valence: 0.327
Acousticness: 0.523
Liveness: 0.0957
Danceability: 0.476 
Energy: 0.242 
Instrumentalness: 0.797 
Speechiness: 0.113 
---
Nombre de la canción: Star Trail
Artista: Kaspa.
Género: lo-fi jazzhop
Valence: 0.0397
Acousticness: 0.945
Liveness: 0.0802
Danceability: 0.593 
Energy: 0.163 
Instrumentalness: 0.42 
Speechiness: 0.0448 
---
Nombre de la canción: Empty Shelves
Artista: Team Astro
Género: lo-fi study
Valence: 0.244
Acousticness: 0.64
Liveness: 0.374
Danceability: 0.725 
Energy: 0.219 


In [ ]:
playlist_name = "Metal"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)
    track_ids = track_ids[:50]

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: Hard Rock /Metal (ID: 1GXRoQWlxTNQiMNkOe7RqA)

IDs de canciones en la playlist seleccionada:
['2pxAohyJptQWTQ5ZRWYijN', '33c2SurTuBpaWAW9YKlvqC', '1jzDzZWeSDBg5fhNc3tczV', '5NIPsWpDjJTFBoPxCUUeXp', '1RQwcsdHIW5p86cx8zuf1I', '61dTqhd46yMkSWmC5LAh5F', '4wBxOGbbVIVkCJGCM5zwVo', '3HzWxmvpQU3QHQ59zw1X4V', '47vDLYqhLl0CaAiqBfvtZE', '1Ug98CRiQYu7wiSym4uLY4', '1hqrYSqvNc9x3BETX1cZhk', '7bRAfjkvqFw2koYKswugoA', '6fH3I6Q2zutJ3rUiygm1lI', '0TI8TP4FitVPoEHPTySx48', '5Q41NLTmGbVPozwHKK7bk2', '2dXsILW8gzkosqleHAvl0v', '7txxAtOMwLLnQTpKeBL6bp', '01Mpj13vURSO3cCLprPt5T', '2FY3V309Dc0X6VS8OWFxC9', '2jz6rWGqVCYsymwxGoaON4', '0W35nxtHtFlseSojmygEsf', '5nekfiTN45vlxG0eNJQQye', '77aoiMh0N8Wgb42RuU14CW', '5X0bLSho0KTiSQYCI0IOcl', '0oynmtmp5en13y77ratUuK', '3ZuVfQriS93y6ofwbIf7lp', '4MhTFsyqIJnjsOweVcU8ug', '4dlupPB3jYjgTuow5ayOkM', '7KcYTtKsLs1JnNYYohHNv0', '41wWrTueWew7qUU0Ru87Mp', '5YU2nvpK4UdpY1WJacIIiw', '5qo8HipgkkuQ9tItiBiTaO', '3cXiUuiGpFmZ8OdAWKk8Ck', '1udmPvJoBnqitbeYKKc9mc', '

In [ ]:
track_id = ['2pxAohyJptQWTQ5ZRWYijN', '33c2SurTuBpaWAW9YKlvqC', '1jzDzZWeSDBg5fhNc3tczV', '5NIPsWpDjJTFBoPxCUUeXp', '1RQwcsdHIW5p86cx8zuf1I', '61dTqhd46yMkSWmC5LAh5F', '4wBxOGbbVIVkCJGCM5zwVo', '3HzWxmvpQU3QHQ59zw1X4V', '47vDLYqhLl0CaAiqBfvtZE', '1Ug98CRiQYu7wiSym4uLY4', '1hqrYSqvNc9x3BETX1cZhk', '7bRAfjkvqFw2koYKswugoA', '6fH3I6Q2zutJ3rUiygm1lI', '0TI8TP4FitVPoEHPTySx48', '5Q41NLTmGbVPozwHKK7bk2', '2dXsILW8gzkosqleHAvl0v', '7txxAtOMwLLnQTpKeBL6bp', '01Mpj13vURSO3cCLprPt5T', '2FY3V309Dc0X6VS8OWFxC9', '2jz6rWGqVCYsymwxGoaON4', '0W35nxtHtFlseSojmygEsf', '5nekfiTN45vlxG0eNJQQye', '77aoiMh0N8Wgb42RuU14CW', '5X0bLSho0KTiSQYCI0IOcl', '0oynmtmp5en13y77ratUuK', '3ZuVfQriS93y6ofwbIf7lp', '4MhTFsyqIJnjsOweVcU8ug', '4dlupPB3jYjgTuow5ayOkM', '7KcYTtKsLs1JnNYYohHNv0', '41wWrTueWew7qUU0Ru87Mp', '5YU2nvpK4UdpY1WJacIIiw', '5qo8HipgkkuQ9tItiBiTaO', '3cXiUuiGpFmZ8OdAWKk8Ck', '1udmPvJoBnqitbeYKKc9mc', '5uRN4EmPA7BWs5f4xsIl0Z', '7vAwjvJs8YSlHcykJu7nc0', '3m5dLMa3z9J87sQdDjTXxt', '4ggy61YkrNendRHDi8fmNF', '4aYLAF6ckQ5ooGGGM7sWAa', '2CzKVfVA9U7OKbIpAVPZOk', '0gGJO7JM51uAUu7EE2lxx6', '3dcvwpAqj1kNOiXzbloC1n', '5H0eyUBNPS8grsIiDlVoPu', '7vE1qmmpaWt6Z5caFGBObe', '3xqzMciySK1wh8oUFbmtPO', '5IX4TbIR5mMHGE4wiWwKW0', '7fSOpoZTwCmS8FON3g5M4O', '0mfbspA6GdxREXA2rCvoyp', '4uzfOKj6HMmLxu2qvmT43E', '0nLiqZ6A27jJri2VCalIUs']
for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']} ")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.1)

Nombre de la canción: The Trooper - 1998 Remastered Version
Artista: Iron Maiden
Género: alternative metal
Valence: 0.696
Acousticness: 0.0238
Liveness: 0.487
Danceability: 0.286 
Energy: 0.856 
Instrumentalness: 0.0542 
Speechiness: 0.0585 
---
Nombre de la canción: Master Of Puppets
Artista: Metallica
Género: hard rock
Valence: 0.588
Acousticness: 0.00161
Liveness: 0.0983
Danceability: 0.541 
Energy: 0.738 
Instrumentalness: 0.194 
Speechiness: 0.0346 
---
Nombre de la canción: Paranoid
Artista: Black Sabbath
Género: album rock
Valence: 0.376
Acousticness: 7.85e-05
Liveness: 0.175
Danceability: 0.429 
Energy: 0.631 
Instrumentalness: 0.00411 
Speechiness: 0.0769 
---
Nombre de la canción: Welcome To The Jungle
Artista: Guns N' Roses
Género: glam metal
Valence: 0.336
Acousticness: 0.0326
Liveness: 0.317
Danceability: 0.445 
Energy: 0.955 
Instrumentalness: 0.284 
Speechiness: 0.0856 
---
Nombre de la canción: Kickstart My Heart
Artista: Mötley Crüe
Género: album rock
Valence: 0.238
Ac

In [ ]:
playlist_name = "Hip Hop rap"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)
    track_ids = track_ids[:50]

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: Hip Hop Mix (ID: 37i9dQZF1EQnqst5TRi17F)

IDs de canciones en la playlist seleccionada:
['6ie0uyyvOKTTuIFBMPiNIl', '6gBFPUFcJLzWGx4lenP6h2', '0io16MKpbeDIdYzmGpQaES', '0VgkVdmE4gld66l8iyGjgx', '7ycWLEP1GsNjVvcjawXz3z', '1lOe9qE0vR9zwWQAOk6CoO', '4LwU4Vp6od3Sb08CsP99GC', '3eekarcy7kvN4yt5ZFzltW', '561jH07mF1jHuk7KlaeF0s', '4X5f3vT8MRuXF68pfjNte5', '78QR3Wp35dqAhFEc2qAGjE', '5Tbpp3OLLClPJF8t1DmrFD', '7KXjTSCq5nL1LoYtL7XAwS', '1CDQzbCz4KSQxHe7LMEgRM', '7floNISpH8VF4z4459Qo18', '0e7ipj03S05BNilyu5bRzt', '503OTo2dSqe7qk76rgsbep', '6DCZcSspjsKoFjzjrWoCdn', '7FIWs0pqAYbP91WWM0vlTQ', '4fukZMkYV1JgJ68NegVLtN', '5sNESr6pQfIhL3krM8CtZn', '2N3U8uTYhjX2KwGT0pf5x0', '5ry2OE6R2zPQFDO85XkgRb', '7lQ8MOhq6IN2w8EYcFNSUk', '0Ed7MeXx64f6OcIuoTRCg1', '4VQNCzfZ3MdHEwwErNXpBo', '2auNjgM4xIOANxtZT4Xe9z', '4mIHgQ3ofK2RK34UdbqMbe', '3yfqSUWxFvZELEM4PmlwIR', '2Ih217RCGAmyQR68Nn7Cqo', '7iXF2W9vKmDoGAhlHdpyIa', '466cKvZn1j45IpxDdYZqdA', '2kQuhkFX7uSVepCD3h29g5', '7GX5flRQZVHRAGd6B4TmDO', '0NzNK

In [ ]:
track_id = ['6ie0uyyvOKTTuIFBMPiNIl', '6gBFPUFcJLzWGx4lenP6h2', '0io16MKpbeDIdYzmGpQaES', '0VgkVdmE4gld66l8iyGjgx', '7ycWLEP1GsNjVvcjawXz3z', '1lOe9qE0vR9zwWQAOk6CoO', '4LwU4Vp6od3Sb08CsP99GC', '3eekarcy7kvN4yt5ZFzltW', '561jH07mF1jHuk7KlaeF0s', '4X5f3vT8MRuXF68pfjNte5', '78QR3Wp35dqAhFEc2qAGjE', '5Tbpp3OLLClPJF8t1DmrFD', '7KXjTSCq5nL1LoYtL7XAwS', '1CDQzbCz4KSQxHe7LMEgRM', '7floNISpH8VF4z4459Qo18', '0e7ipj03S05BNilyu5bRzt', '503OTo2dSqe7qk76rgsbep', '6DCZcSspjsKoFjzjrWoCdn', '7FIWs0pqAYbP91WWM0vlTQ', '4fukZMkYV1JgJ68NegVLtN', '5sNESr6pQfIhL3krM8CtZn', '2N3U8uTYhjX2KwGT0pf5x0', '5ry2OE6R2zPQFDO85XkgRb', '7lQ8MOhq6IN2w8EYcFNSUk', '0Ed7MeXx64f6OcIuoTRCg1', '4VQNCzfZ3MdHEwwErNXpBo', '2auNjgM4xIOANxtZT4Xe9z', '4mIHgQ3ofK2RK34UdbqMbe', '3yfqSUWxFvZELEM4PmlwIR', '2Ih217RCGAmyQR68Nn7Cqo', '7iXF2W9vKmDoGAhlHdpyIa', '466cKvZn1j45IpxDdYZqdA', '2kQuhkFX7uSVepCD3h29g5', '7GX5flRQZVHRAGd6B4TmDO', '0NzNKU2MJ9LCetT2uZMJH2', '2tudvzsrR56uom6smgOcSf', '1SWVDBtw6h3tm9OehOkDhv', '2G7V7zsVDxg1yRsu7Ew9RJ', '4fv1OzrStC17Mqck9LGldk', '6LwrEZNkvFTOypNwSLjuaJ', '6vN77lE9LK6HP2DewaN6HZ', '3a1lNhkSLSkpJE4MSHpDu9', '3F5CgOj3wFlRv51JsHbxhe', '78JKJfKsqgeBDBF58gv1SF', '7Ie9W94M7OjPoZVV216Xus', '2JvzF1RMd7lE3KmFlsyZD8', '7dt6x5M1jzdTEt8oCbisTK', '0TlLq3lA83rQOYtrqBqSct', '3ia3dJETSOllPsv3LJkE35', '2XHzzp1j4IfTNp1FTn7YFg']
for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']} ")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.1)

Nombre de la canción: Judge Judy
Artista: Tyler, The Creator
Género: hip hop
Valence: 0.362
Acousticness: 0.297
Liveness: 0.165
Danceability: 0.622 
Energy: 0.661 
Instrumentalness: 0.000135 
Speechiness: 0.0826 
---
Nombre de la canción: goosebumps
Artista: Travis Scott
Género: rap
Valence: 0.43
Acousticness: 0.0847
Liveness: 0.149
Danceability: 0.841 
Energy: 0.728 
Instrumentalness: 0 
Speechiness: 0.0484 
---
Nombre de la canción: Embrace It
Artista: Ndotz
Género: chill drill
Valence: 0.788
Acousticness: 0.475
Liveness: 0.057
Danceability: 0.882 
Energy: 0.645 
Instrumentalness: 4.13e-05 
Speechiness: 0.0601 
---
Nombre de la canción: Mask Off
Artista: Future
Género: atl hip hop
Valence: 0.281
Acousticness: 0.0102
Liveness: 0.165
Danceability: 0.833 
Energy: 0.434 
Instrumentalness: 0.0219 
Speechiness: 0.431 
---
Nombre de la canción: Praise The Lord (Da Shine) (feat. Skepta)
Artista: A$AP Rocky
Género: east coast hip hop
Valence: 0.294
Acousticness: 0.0609
Liveness: 0.1
Danceabil

In [ ]:
playlist_name = "Americans Rap 2000s"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)
    track_ids = track_ids[:50]

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: Américain Rap 🇺🇸 90-2000 (ID: 7K5T1por7Awge7QiukTFfS)

IDs de canciones en la playlist seleccionada:
['4AE7Lj39VnSZNOmGH2iZaq', '5D2mYZuzcgjpchVY1pmTPh', '0KbIwqCAHmXWu7hrszXY6j', '2xTft6GEZeTyWNpdX94rkf', '0Ed7MeXx64f6OcIuoTRCg1', '70ludZZEUJy1aFVkPI0Kpi', '3oairMMtNVnUppKwroxou4', '4PmMVdIFpJTB9WQbmILf4p', '5XhkV07Vou38wnrzwURUOC', '7KwZNVEaqikRSBSpyhXK2j', '1FK0sZfZzCAIDUn7htCemp', '5osSw3tL07Tuid7AWsvYcc', '2n7h09QrGitVul66b2D8Re', '5kGsDi86P9pZux4x4wopHV', '4hBy3nZVzQV4F3jxe1wWL1', '1RjU7UhRICmdFPKxCucLgq', '2212oBX0YELcs0sr6DqGer', '7iL6o9tox1zgHpKUfh9vuC', '62d33ZVLAxgmlZnofs7Ang', '10C83Ul1IxCyuoJRWtx5GN', '3ssX20QT5c3nA9wk78V1LQ', '3UOuBNEin5peSRqdzvlnWM', '3NGT0Td7H4Is1qrlDQJxma', '33ZXjLCpiINn8eQIDYEPTD', '3G0yz3DZn3lfraledmBCT0', '7lQWRAjyhTpCWFC0jmclT4', '4IYKjN1DrYzxKXt0umJqsG', '09EwNbGvUyu7BDEYG0cJro', '5ngSGTy4Tw8cIBCMeIgoc9', '6mcu7D7QuABVwUGDwovOEh', '2eknDuc5IeulN0hnwEZsjF', '69bHJ9qs5FrUJbKP8xU8uZ', '5rb9QrpfcKFHM1EUbSIurX', '2g8HN35AnVGIk7B8yM

In [ ]:
track_id = ['4AE7Lj39VnSZNOmGH2iZaq', '5D2mYZuzcgjpchVY1pmTPh', '0KbIwqCAHmXWu7hrszXY6j', '2xTft6GEZeTyWNpdX94rkf', '0Ed7MeXx64f6OcIuoTRCg1', '70ludZZEUJy1aFVkPI0Kpi', '3oairMMtNVnUppKwroxou4', '4PmMVdIFpJTB9WQbmILf4p', '5XhkV07Vou38wnrzwURUOC', '7KwZNVEaqikRSBSpyhXK2j', '1FK0sZfZzCAIDUn7htCemp', '5osSw3tL07Tuid7AWsvYcc', '2n7h09QrGitVul66b2D8Re', '5kGsDi86P9pZux4x4wopHV', '4hBy3nZVzQV4F3jxe1wWL1', '1RjU7UhRICmdFPKxCucLgq', '2212oBX0YELcs0sr6DqGer', '7iL6o9tox1zgHpKUfh9vuC', '62d33ZVLAxgmlZnofs7Ang', '10C83Ul1IxCyuoJRWtx5GN', '3ssX20QT5c3nA9wk78V1LQ', '3UOuBNEin5peSRqdzvlnWM', '3NGT0Td7H4Is1qrlDQJxma', '33ZXjLCpiINn8eQIDYEPTD', '3G0yz3DZn3lfraledmBCT0', '7lQWRAjyhTpCWFC0jmclT4', '4IYKjN1DrYzxKXt0umJqsG', '09EwNbGvUyu7BDEYG0cJro', '5ngSGTy4Tw8cIBCMeIgoc9', '6mcu7D7QuABVwUGDwovOEh', '2eknDuc5IeulN0hnwEZsjF', '69bHJ9qs5FrUJbKP8xU8uZ', '5rb9QrpfcKFHM1EUbSIurX', '2g8HN35AnVGIk7B8yMucww', '13XHuE00ElL5thSxkaEXxK', '4E5IFAXCob6QqZaJMTw5YN', '0WKYRFtH6KKbaNWjsxqm70', '7N1Vjtzr1lmmCW9iasQ8YO', '2cGJxXXIBrxMhk2fWUJTLs', '0bp6rlpAjUnuIqQfmLs8C0', '4rLpAOk5KVNcqEMK3m6Lje', '3lcC9CnN7Rpo1ugATzmYjY', '61LtVmmkGr8P9I2tSPvdpf', '3n4cCOBMqolImjZXPwWg1f', '3CZ31XPvscItyfSNf1ox0M', '00i2HU7TEzzftShjRrDSEF', '4voEoczU7Ijborps9XF1n3', '7jLbTp3qZzah9kMIdW8e5M', '2Z6I3youZAlfRvsyzJAGQA', '1SWVDBtw6h3tm9OehOkDhv']
for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']} ")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.1)

Nombre de la canción: Do For Love
Artista: 2Pac
Género: g funk
Valence: 0.717
Acousticness: 0.0578
Liveness: 0.0582
Danceability: 0.874 
Energy: 0.63 
Instrumentalness: 0.00479 
Speechiness: 0.298 
---
Nombre de la canción: Candy Shop
Artista: 50 Cent
Género: east coast hip hop
Valence: 0.755
Acousticness: 0.0253
Liveness: 0.38
Danceability: 0.613 
Energy: 0.574 
Instrumentalness: 3.13e-05 
Speechiness: 0.467 
---
Nombre de la canción: Party We Will Throw Now!
Artista: Warren G
Género: g funk
Valence: 0.712
Acousticness: 0.0476
Liveness: 0.0608
Danceability: 0.934 
Energy: 0.562 
Instrumentalness: 1.31e-06 
Speechiness: 0.24 
---
Nombre de la canción: All Eyez On Me
Artista: 2Pac
Género: g funk
Valence: 0.858
Acousticness: 0.03
Liveness: 0.333
Danceability: 0.873 
Energy: 0.719 
Instrumentalness: 0 
Speechiness: 0.248 
---
Nombre de la canción: Xxplosive
Artista: Dr. Dre
Género: g funk
Valence: 0.929
Acousticness: 0.176
Liveness: 0.139
Danceability: 0.765 
Energy: 0.88 
Instrumentalnes

In [ ]:
playlist_name = "EMD"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)
    track_ids = track_ids[:50]

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: EDMR Bilateral Music 🎶🎵 (ID: 72jtliEwhdRx2AvIX1R68A)

IDs de canciones en la playlist seleccionada:
['1ug0lXizpH928IqA56Reks', '2Lik7eYnsSB2fPn0k2VAeR', '6XYGQXaMxYjUA3R5EBx4Jc', '1FVjAPYGiIlFRkFWsAr0Nw', '14HXHkcrt2gMAEN3lQb8V4', '4NYuPUFTr6wmKgh5UioLq0', '5P6tGPxqUNRfxj3AliUlN7', '0Ye9dr9k8wsM3gv0yPOjuj', '2ltejgkcrF3XmXMMI5HTPY', '6pYvrAtxHb2wwhmYtOSYlp', '28liZoErnAlWS66JhL3ZM2', '1qGf0g6ZZEBt8Z7U9HVgoF', '4BgJcNg318AfGmb5Ny1zOw', '7cEjzxbTY9yOsVspJOpwme', '13vU2lqVVClRBAvDOAKr5L', '42m8Ct1bJ8xxQ9Xn407s8K', '3sydV8eodOZbqlyvZSGTzE', '64z8vXpE2pxeD5hdGq3HGJ', '1lAwje1MCr6B5E5polZ9Wy', '5ZfQjYCKtLZZR7e5PAFTgM', '6zy2OarpM5gezVeWas9838', '5M6eaG4cUnGAcbMSyYzjbH', '3cTRiVedT0Z0pvdpicJ7a5', '0Tmpm1T6b81ZG6wpLNCCnW', '6sKhUgi3yS3owbEyvH6gKp', '6lOFP1Haek1QFIWm3zcA0a']


In [ ]:
track_id =['1ug0lXizpH928IqA56Reks', '2Lik7eYnsSB2fPn0k2VAeR', '6XYGQXaMxYjUA3R5EBx4Jc', '1FVjAPYGiIlFRkFWsAr0Nw', '14HXHkcrt2gMAEN3lQb8V4', '4NYuPUFTr6wmKgh5UioLq0', '5P6tGPxqUNRfxj3AliUlN7', '0Ye9dr9k8wsM3gv0yPOjuj', '2ltejgkcrF3XmXMMI5HTPY', '6pYvrAtxHb2wwhmYtOSYlp', '28liZoErnAlWS66JhL3ZM2', '1qGf0g6ZZEBt8Z7U9HVgoF', '4BgJcNg318AfGmb5Ny1zOw', '7cEjzxbTY9yOsVspJOpwme', '13vU2lqVVClRBAvDOAKr5L', '42m8Ct1bJ8xxQ9Xn407s8K', '3sydV8eodOZbqlyvZSGTzE', '64z8vXpE2pxeD5hdGq3HGJ', '1lAwje1MCr6B5E5polZ9Wy', '5ZfQjYCKtLZZR7e5PAFTgM', '6zy2OarpM5gezVeWas9838', '5M6eaG4cUnGAcbMSyYzjbH', '3cTRiVedT0Z0pvdpicJ7a5', '0Tmpm1T6b81ZG6wpLNCCnW', '6sKhUgi3yS3owbEyvH6gKp', '6lOFP1Haek1QFIWm3zcA0a']
for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']} ")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.1)

Nombre de la canción: Strength Within
Artista: EMDR
Género: Unknown
Valence: 0.0361
Acousticness: 0.958
Liveness: 0.147
Danceability: 0.175 
Energy: 0.0347 
Instrumentalness: 0.97 
Speechiness: 0.0413 
---
Nombre de la canción: Bilateral Stimulation
Artista: Mark Grant
Género: Unknown
Valence: 0.0374
Acousticness: 0.202
Liveness: 0.0698
Danceability: 0.618 
Energy: 0.436 
Instrumentalness: 0.973 
Speechiness: 0.0982 
---
Nombre de la canción: Breathing Space
Artista: Sacred Earth
Género: healing
Valence: 0.0307
Acousticness: 0.979
Liveness: 0.0908
Danceability: 0.0612 
Energy: 0.017 
Instrumentalness: 0.974 
Speechiness: 0.0496 
---
Nombre de la canción: There Is a River
Artista: Bryan Cumming
Género: Unknown
Valence: 0.405
Acousticness: 0.822
Liveness: 0.943
Danceability: 0.537 
Energy: 0.298 
Instrumentalness: 0.921 
Speechiness: 0.0354 
---
Nombre de la canción: Inner Light
Artista: EMDR
Género: Unknown
Valence: 0.038
Acousticness: 0.883
Liveness: 0.108
Danceability: 0.146 
Energy: 

In [ ]:
playlist_name = "Kpop"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)
    track_ids = track_ids[:50]

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: K-Pop Mix (ID: 37i9dQZF1EQpesGsmIyqcW)

IDs de canciones en la playlist seleccionada:
['5vNRhkKd0yEAg8suGBpjeY', '2CspwnypzT7rcWI9RfsoSb', '6vvPecFTmWxDfEJ6cYT1wa', '5XWlyfo0kZ8LF7VSyfS4Ew', '1aKvZDoLGkNMxoRYgkckZG', '2HRgqmZQC0MC7GeNuDIXHN', '2wVDWtLKXunswWecARNILj', '7hU3IHwjX150XLoTVmjD0q', '3dbLT62Cvs46Ju7a8gpr36', '4SFknyjLcyTLJFPKD2m96o', '6aJn7Cst74cj4lNIiPRgav', '2dHoVW9AxJVSRebPRyV2aA', '3yDRcs0Y4pPzkvMbUfeF9H', '1SS0WlKhJewviwEDZ6dWj0', '6hvczQ05jc1yGlp9zhb95V', '5sdQOyqq2IDhvmx2lHOpwd', '2KslE17cAJNHTsI2MI0jb2', '7uQZVznj0uQOGC9KhV2Mg6', '3lwK3WyKAI5bUuGY1OgGbZ', '4Ws314Ylb27BVsvlZOy30C', '65FftemJ1DbbZ45DUfHJXE', '2pn8dNVSpYnAtlKFC8Q0DJ', '5elW2CKSoqjYoJ32AGDxf1', '1njlnn8ZKHI77Pe9szIONR', '2kIUILBPlz4exX9xIFS275', '01qFKNWq73UfEslI0GvumE', '5zwwW9Oq7ubSxoCGyW1nbY', '21aOLk12MksET8AsbU0SI6', '4a9tbd947vo9K8Vti9JwcI', '6gcuJpHu0Ey30D5WR76y98', '69CrOS7vEHIrhC2ILyEi0s', '4lQsB3ERTWSNaAN1IkuNRl', '3aAnwyBJY9OLNLqSgd4fZU', '2EmcTFQ1rM11wp2ztsXTHa', '38tXZcL

In [ ]:
track_id = ['5vNRhkKd0yEAg8suGBpjeY', '2CspwnypzT7rcWI9RfsoSb', '6vvPecFTmWxDfEJ6cYT1wa', '5XWlyfo0kZ8LF7VSyfS4Ew', '1aKvZDoLGkNMxoRYgkckZG', '2HRgqmZQC0MC7GeNuDIXHN', '2wVDWtLKXunswWecARNILj', '7hU3IHwjX150XLoTVmjD0q', '3dbLT62Cvs46Ju7a8gpr36', '4SFknyjLcyTLJFPKD2m96o', '6aJn7Cst74cj4lNIiPRgav', '2dHoVW9AxJVSRebPRyV2aA', '3yDRcs0Y4pPzkvMbUfeF9H', '1SS0WlKhJewviwEDZ6dWj0', '6hvczQ05jc1yGlp9zhb95V', '5sdQOyqq2IDhvmx2lHOpwd', '2KslE17cAJNHTsI2MI0jb2', '7uQZVznj0uQOGC9KhV2Mg6', '3lwK3WyKAI5bUuGY1OgGbZ', '4Ws314Ylb27BVsvlZOy30C', '65FftemJ1DbbZ45DUfHJXE', '2pn8dNVSpYnAtlKFC8Q0DJ', '5elW2CKSoqjYoJ32AGDxf1', '1njlnn8ZKHI77Pe9szIONR', '2kIUILBPlz4exX9xIFS275', '01qFKNWq73UfEslI0GvumE', '5zwwW9Oq7ubSxoCGyW1nbY', '21aOLk12MksET8AsbU0SI6', '4a9tbd947vo9K8Vti9JwcI', '6gcuJpHu0Ey30D5WR76y98', '69CrOS7vEHIrhC2ILyEi0s', '4lQsB3ERTWSNaAN1IkuNRl', '3aAnwyBJY9OLNLqSgd4fZU', '2EmcTFQ1rM11wp2ztsXTHa', '38tXZcL1gZRfbqfOG0VMTH', '2O4Bb2WCkjlTPO827OnBMI', '4JUPEh2DVSXFGExu4Uxevz', '7mpdNiaQvygj2rHoxkzMfa', '5QDLhrAOJJdNAmCTJ8xMyW', '0a4MMyCrzT0En247IhqZbD', '74X2u8JMVooG2QbjRxXwR8', '7gRFDGEzF9UkBV233yv2dc', '7tI8dRuH2Yc6RuoTjxo4dU', '6jjYDGxVJsWS0a5wlVF5vS', '2YmfV4lAjrAQvuggKCUX6m', '3lOMJTQTd6J34faYwASc33', '4ZxOuNHhpyOj4gv52MtQpT', '3VBj0lzjmhTzVFPEDOjNCG', '5FVbvttjEvQ8r2BgUcJgNg', '3r8RuvgbX9s7ammBn07D3W']
for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']} ")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.1)

Nombre de la canción: APT.
Artista: ROSÉ
Género: k-pop
Valence: 0.939
Acousticness: 0.0283
Liveness: 0.355
Danceability: 0.777 
Energy: 0.783 
Instrumentalness: 0 
Speechiness: 0.26 
---
Nombre de la canción: Mantra
Artista: JENNIE
Género: k-pop
Valence: 0.838
Acousticness: 0.00434
Liveness: 0.0815
Danceability: 0.894 
Energy: 0.725 
Instrumentalness: 1.15e-05 
Speechiness: 0.0432 
---
Nombre de la canción: Rockstar
Artista: LISA
Género: k-pop
Valence: 0.568
Acousticness: 0.129
Liveness: 0.0293
Danceability: 0.668 
Energy: 0.628 
Instrumentalness: 0 
Speechiness: 0.046 
---
Nombre de la canción: Drama
Artista: aespa
Género: k-pop girl group
Valence: 0.378
Acousticness: 0.164
Liveness: 0.363
Danceability: 0.644 
Energy: 0.872 
Instrumentalness: 0 
Speechiness: 0.0417 
---
Nombre de la canción: Magnetic
Artista: ILLIT
Género: 5th gen k-pop
Valence: 0.69
Acousticness: 0.257
Liveness: 0.0876
Danceability: 0.797 
Energy: 0.668 
Instrumentalness: 0 
Speechiness: 0.0687 
---
Nombre de la canc

In [ ]:
playlist_name = "rhythm and blues"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)
    track_ids = track_ids[:50]

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: Rythm and Blues classics (ID: 7lHRZpTSYXk2hY6wpzQvXX)

IDs de canciones en la playlist seleccionada:
['7gmyxUiWkGVMu6aNA2P7F4', '4yQw7FR9lcvL6RHtegbJBh', '3odYOpssnLbn6TnzdEst2R', '39Xb2mnMH7w8rpg5S3u9FS', '0HHswZ0tJX8O5YBNoZwoFl', '48VGtH57hg82VXX8s39LLK', '533NW3XHuW3MjO4vLqY2iV', '58gMJ03X0OHzpUlioZ2oCn', '3cLyEer3Ek7V3nPocOWWAn', '7aKmiNpiwWXPKIdBkPlIWL', '4Y13tcnF3rc1SHfNyVTjAG', '5GLr9bkV2czl8o7zJoWXca', '3uqZzh3SSuNLhWP1zSWjxv', '4OWOsMAaqPPMrzJhZMxK2N', '2pMwweGkjNLXZkSDZ63LYf', '2f1kaz9iKDa9aHwZMfdbti', '4eGHlplaq1ME8oetnTuFFf', '76ICmoJ4PcoMWoooaTxnQs', '0pvA063uNyGQFp7DNtmk0P', '7s25THrKz86DM225dOYwnr', '6rLqjzGV5VMLDWEnuUqi8q', '4591VqUIXysNlmI5NcAIUd', '6PgVDY8GTkxF3GmhVGPzoB', '2G2YzndIA6jeWFPBXhUjh5', '4fQMGlCawbTkH9yPPZ49kP', '3ZxfehzRtB3oJcBuxWXFZQ', '4NL8D4RQsC6ux6eI3m3Lg5', '2xar08Fq5xra2KKZs5Bw9j', '63xdwScd1Ai1GigAwQxE8y', '6bBwFPgVdWxuc4uWEyALCa', '5YLnfy7R2kueN0BRPkjiEG', '3VgrSJEKBRT3xmwzpaxxEF', '62r1CS9zL1FL5Ilb7hLX7n', '1P67Uf3XJuWISCsQF2

In [ ]:
track_id = ['7gmyxUiWkGVMu6aNA2P7F4', '4yQw7FR9lcvL6RHtegbJBh', '3odYOpssnLbn6TnzdEst2R', '39Xb2mnMH7w8rpg5S3u9FS', '0HHswZ0tJX8O5YBNoZwoFl', '48VGtH57hg82VXX8s39LLK', '533NW3XHuW3MjO4vLqY2iV', '58gMJ03X0OHzpUlioZ2oCn', '3cLyEer3Ek7V3nPocOWWAn', '7aKmiNpiwWXPKIdBkPlIWL', '4Y13tcnF3rc1SHfNyVTjAG', '5GLr9bkV2czl8o7zJoWXca', '3uqZzh3SSuNLhWP1zSWjxv', '4OWOsMAaqPPMrzJhZMxK2N', '2pMwweGkjNLXZkSDZ63LYf', '2f1kaz9iKDa9aHwZMfdbti', '4eGHlplaq1ME8oetnTuFFf', '76ICmoJ4PcoMWoooaTxnQs', '0pvA063uNyGQFp7DNtmk0P', '7s25THrKz86DM225dOYwnr', '6rLqjzGV5VMLDWEnuUqi8q', '4591VqUIXysNlmI5NcAIUd', '6PgVDY8GTkxF3GmhVGPzoB', '2G2YzndIA6jeWFPBXhUjh5', '4fQMGlCawbTkH9yPPZ49kP', '3ZxfehzRtB3oJcBuxWXFZQ', '4NL8D4RQsC6ux6eI3m3Lg5', '2xar08Fq5xra2KKZs5Bw9j', '63xdwScd1Ai1GigAwQxE8y', '6bBwFPgVdWxuc4uWEyALCa', '5YLnfy7R2kueN0BRPkjiEG', '3VgrSJEKBRT3xmwzpaxxEF', '62r1CS9zL1FL5Ilb7hLX7n', '1P67Uf3XJuWISCsQF25292', '71tjsDvB4EMJqNG8EMmFnb']
for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']} ")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.1)

Nombre de la canción: Knock on Wood
Artista: Eddie Floyd
Género: memphis soul
Valence: 0.958
Acousticness: 0.446
Liveness: 0.0475
Danceability: 0.841 
Energy: 0.65 
Instrumentalness: 0.0463 
Speechiness: 0.0369 
---
Nombre de la canción: Think
Artista: Aretha Franklin
Género: classic soul
Valence: 0.926
Acousticness: 0.251
Liveness: 0.221
Danceability: 0.687 
Energy: 0.63 
Instrumentalness: 0 
Speechiness: 0.0389 
---
Nombre de la canción: I Thank You
Artista: Sam & Dave
Género: classic soul
Valence: 0.62
Acousticness: 0.0449
Liveness: 0.0989
Danceability: 0.877 
Energy: 0.611 
Instrumentalness: 1.05e-05 
Speechiness: 0.0737 
---
Nombre de la canción: Harlem Shuffle
Artista: Bob & Earl
Género: classic soul
Valence: 0.766
Acousticness: 0.55
Liveness: 0.0584
Danceability: 0.646 
Energy: 0.804 
Instrumentalness: 0.0484 
Speechiness: 0.0426 
---
Nombre de la canción: Stagger Lee
Artista: Lloyd Price
Género: doo-wop
Valence: 0.787
Acousticness: 0.727
Liveness: 0.229
Danceability: 0.382 
Ene

In [ ]:
playlist_name = "Country"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)
    track_ids = track_ids[:50]

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: COUNTRY HITS 2024 🔥 New Country Songs + Top Hits (ID: 4Jb4PDWREzNnbZcOHPcZPy)

IDs de canciones en la playlist seleccionada:
['5IZXB5IKAD2qlvTPJYDCFB', '2uqYupMHANxnwgeiXTZXzd', '3Rfre3qkrhwdZZ7dyznwbN', '2FQrifJ1N335Ljm3TjTVVf', '6StwwqB84sJeLr7tZDTxEX', '4pkb8SbRGeHAvdb87v9rpf', '7hR22TOX3RorxJPcsz5Wbo', '0llPOBVoJYDAtdmVlNE41A', '6GG4yyk3UATdBfTHVgI8PB', '6tWGe6nvXFWT0KhGCXM89q', '4qGyXDT3rHEW6d2HP1otDj', '5ya0TmUQw2wHMkq36rPsnd', '7fveJ3pk1eSfxBdVkzdXZ0', '1SBoKG4MjtH2vMUpftdQtc', '5JRYVSZ6K2jY0kznYBmoqr', '18ztlZ4mfETc4BUbLRrHbT', '36wAwEkSDpNQ4oQeRLYC3L', '7lCs3stpWCtAn5Y2lmnPhV', '78DcrUOhBhVod09vFiXn7P', '61SRKyox0R9jCzci4JXKBS', '0RBhO45NyARwVqqLQBZKwn', '0G3fI2EvhLAtBKZ66Tow9V', '1FhgmMvNlh3nJ1Ldc2eE6H', '1osfLqL6L2iQsirRf83ded', '1A0BUe8KbR1TCWPRZjwJnA', '1Lo0QY9cvc8sUB2vnIOxDT', '59uQI0PADDKeE6UZDTJEe8', '2z5t8IRRtt5vwkSzP9umJo', '6fX8O20VdjUBCGYZbGyq6c', '4JEPMdMizbAWlaCtLaTjvT', '4KULAymBBJcPRpk1yO4dOG', '0hQyFpC8af9Dj89fEYkXPG', '67IfTiMwPNN0wQE7Scfz

In [ ]:
track_id = ['5IZXB5IKAD2qlvTPJYDCFB', '2uqYupMHANxnwgeiXTZXzd', '3Rfre3qkrhwdZZ7dyznwbN', '2FQrifJ1N335Ljm3TjTVVf', '6StwwqB84sJeLr7tZDTxEX', '4pkb8SbRGeHAvdb87v9rpf', '7hR22TOX3RorxJPcsz5Wbo', '0llPOBVoJYDAtdmVlNE41A', '6GG4yyk3UATdBfTHVgI8PB', '6tWGe6nvXFWT0KhGCXM89q', '4qGyXDT3rHEW6d2HP1otDj', '5ya0TmUQw2wHMkq36rPsnd', '7fveJ3pk1eSfxBdVkzdXZ0', '1SBoKG4MjtH2vMUpftdQtc', '5JRYVSZ6K2jY0kznYBmoqr', '18ztlZ4mfETc4BUbLRrHbT', '36wAwEkSDpNQ4oQeRLYC3L', '7lCs3stpWCtAn5Y2lmnPhV', '78DcrUOhBhVod09vFiXn7P', '61SRKyox0R9jCzci4JXKBS', '0RBhO45NyARwVqqLQBZKwn', '0G3fI2EvhLAtBKZ66Tow9V', '1FhgmMvNlh3nJ1Ldc2eE6H', '1osfLqL6L2iQsirRf83ded', '1A0BUe8KbR1TCWPRZjwJnA', '1Lo0QY9cvc8sUB2vnIOxDT', '59uQI0PADDKeE6UZDTJEe8', '2z5t8IRRtt5vwkSzP9umJo', '6fX8O20VdjUBCGYZbGyq6c', '4JEPMdMizbAWlaCtLaTjvT', '4KULAymBBJcPRpk1yO4dOG', '0hQyFpC8af9Dj89fEYkXPG', '67IfTiMwPNN0wQE7Scfzu9', '0uEg78fi0MPdimE66H8EAk', '3Sg0jiuv2ckmxTBX4tLJQq', '0mNzElhEofvgMWAJoOA4q9', '2eGvz6O6tVnh9nKiitDEhZ', '7wLShogStyDeZvL0a6daN5', '5kB6WithhB8Iv08EMNmAbt', '4qcsbI4duzSGvUoyi5xh5m', '3eNOyXmzuA46bUvi8FkIoW', '1KtQmheytoTI6kOM8yCowf', '6VpH2prT3NGiK5pnpG2L3J', '07on0OB2cdPt79IFsgdRUy', '0bILU0UiNDmA5Ff6whhb2R', '0NJQqsBbSrLysAJOBCgtXb', '6UoKX6uLJwhsnyTp5k5StP', '2PyJAiQjp1OPkow2FJZKHR', '6NdnvPwQvZb147DVa4x14h', '103u49mLoJUge83B1CnLxI']
for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']} ")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.1)

Nombre de la canción: I Had Some Help (Feat. Morgan Wallen)
Artista: Post Malone
Género: dfw rap
Valence: 0.753
Acousticness: 0.0113
Liveness: 0.227
Danceability: 0.644 
Energy: 0.874 
Instrumentalness: 0 
Speechiness: 0.0265 
---
Nombre de la canción: Austin (Boots Stop Workin')
Artista: Dasha
Género: Unknown
Valence: 0.739
Acousticness: 0.464
Liveness: 0.108
Danceability: 0.756 
Energy: 0.672 
Instrumentalness: 3.37e-06 
Speechiness: 0.0522 
---
Nombre de la canción: Lonely Road (with Jelly Roll)
Artista: mgk
Género: ohio hip hop
Valence: 0.235
Acousticness: 0.0103
Liveness: 0.365
Danceability: 0.549 
Energy: 0.679 
Instrumentalness: 0 
Speechiness: 0.0346 
---
Nombre de la canción: A Bar Song (Tipsy)
Artista: Shaboozey
Género: modern country pop
Valence: 0.604
Acousticness: 0.0633
Liveness: 0.0804
Danceability: 0.722 
Energy: 0.709 
Instrumentalness: 0 
Speechiness: 0.0273 
---
Nombre de la canción: Guy For That (Feat. Luke Combs)
Artista: Post Malone
Género: dfw rap
Valence: 0.495


In [ ]:
playlist_name = "Folk y acustica"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)
    track_ids = track_ids[:50]

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: Mix Folk & Acustica (ID: 5eOFc2IZuFtGt6Y2FlqBBX)

IDs de canciones en la playlist seleccionada:
['2M9ro2krNb7nr7HSprkEgo', '6R4pk0zpkyS7KWRXOQkTa2', '6d2MCbByfqhU85Fmj8WCAN', '3bHhUEOTIbezeZ856R0BX5', '476V2d6iA2tWXgQboKmTtA', '1UKobFsdqNXQb8OthimCKe', '5y788ya4NvwhBznoDIcXwK', '6mFkJmJqdDVQ1REhVfGgd1', '3O090PYVo1do7s3PMCoVoz', '4RaxnzmTN4bojbbEYtvXjk', '73C8vVm2BRLLBQ8FH6N6Qm', '6J0bdIZElZMymahgVWRE9B', '1B10XgaxSXRLAFq967oMpF', '6Wz6WuPYs1kOe4clTIkM1q', '7mjSHL2Eb0kAwiKbvNNyD9', '1QJnDJuYOGj05iSc4eWGr5', '4gFakkHz0RjUrbf9Ufilb7', '0YMFcrMtBowDdD5bPz0cgy', '2MAVcVr2oylw2OZ3hojWYj', '5eReKc1D8ZQAg9kzQeHRdH', '5yGVHzJ2Se8pDzR7FIYUlU', '4e2LlObWWxvG0pk4jdbiPy', '0fzCtVM9D5UEwiLqcY8Ouq', '2E97VhTGR4j91H7YtVnxyM', '4h5m56ExHLPUC9NGz7U1Qa', '3g2fYZW5v2od8KIF7VktT0', '2kKNeapwCHUUvMhmAOGce6', '15kLz0zNFdtsaJdXKSGmTU', '0RsMTlG08zUhYSz1nWi1wp', '4LGF2tDg3878bs0mQPByZ4', '3ZLF4Lcg9WvH9cQMTwNPqq', '6Wk6A8k7NjjSsPQT9Z3FXM', '6EHwAGln0IHqtXmO4oMzf6', '4TOMI010Sd4ZAX4aZ5TS85'

In [ ]:
track_id = ['2M9ro2krNb7nr7HSprkEgo', '6R4pk0zpkyS7KWRXOQkTa2', '6d2MCbByfqhU85Fmj8WCAN', '3bHhUEOTIbezeZ856R0BX5', '476V2d6iA2tWXgQboKmTtA', '1UKobFsdqNXQb8OthimCKe', '5y788ya4NvwhBznoDIcXwK', '6mFkJmJqdDVQ1REhVfGgd1', '3O090PYVo1do7s3PMCoVoz', '4RaxnzmTN4bojbbEYtvXjk', '73C8vVm2BRLLBQ8FH6N6Qm', '6J0bdIZElZMymahgVWRE9B', '1B10XgaxSXRLAFq967oMpF', '6Wz6WuPYs1kOe4clTIkM1q', '7mjSHL2Eb0kAwiKbvNNyD9', '1QJnDJuYOGj05iSc4eWGr5', '4gFakkHz0RjUrbf9Ufilb7', '0YMFcrMtBowDdD5bPz0cgy', '2MAVcVr2oylw2OZ3hojWYj', '5eReKc1D8ZQAg9kzQeHRdH', '5yGVHzJ2Se8pDzR7FIYUlU', '4e2LlObWWxvG0pk4jdbiPy', '0fzCtVM9D5UEwiLqcY8Ouq', '2E97VhTGR4j91H7YtVnxyM', '4h5m56ExHLPUC9NGz7U1Qa', '3g2fYZW5v2od8KIF7VktT0', '2kKNeapwCHUUvMhmAOGce6', '15kLz0zNFdtsaJdXKSGmTU', '0RsMTlG08zUhYSz1nWi1wp', '4LGF2tDg3878bs0mQPByZ4', '3ZLF4Lcg9WvH9cQMTwNPqq', '6Wk6A8k7NjjSsPQT9Z3FXM', '6EHwAGln0IHqtXmO4oMzf6', '4TOMI010Sd4ZAX4aZ5TS85', '6vTX4Fc8Zp2YWM3hHUQviF', '6HTVZeVQ9J6Uiq6tHESxU9', '00SzIFHBVTpRonBTHnHuok', '0FwPXCidLeD3JJg9SjqQBE', '1Ip1XhR9OPvA9rPfm8uFZU', '4lxGVzcUaSF5HW5jtWnShV', '3G69vJMWsX6ZohTykad2AU', '4E3Mdw30kJvNznoaoyFJOr', '7zkLpY72g6lKQbiHDqri1S', '0WV1JpJw9mCmEAw0EyudFp', '3t4p9vIhemAhCMr6hfL2pa', '3IsWNivZZHYURT2kTIvBDU', '7thzIJfxiZNZaPFA5OR9s1', '2fDVMNQmFuMORDD3dP7GkN', '1TuxZUwceO5pyXMJ7tdd2R', '2aczN5ldKR3HUFRrz65Sr7']
for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']} ")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.3)

Nombre de la canción: Fast Car
Artista: Tracy Chapman
Género: folk
Valence: 0.194
Acousticness: 0.313
Liveness: 0.131
Danceability: 0.711 
Energy: 0.292 
Instrumentalness: 0 
Speechiness: 0.037 
---
Nombre de la canción: Another Lonely Day
Artista: Ben Harper
Género: neo mellow
Valence: 0.535
Acousticness: 0.775
Liveness: 0.112
Danceability: 0.634 
Energy: 0.138 
Instrumentalness: 0.155 
Speechiness: 0.0427 
---
Nombre de la canción: Salt And The Sea
Artista: The Lumineers
Género: folk-pop
Valence: 0.0712
Acousticness: 0.704
Liveness: 0.128
Danceability: 0.63 
Energy: 0.31 
Instrumentalness: 0.0011 
Speechiness: 0.028 
---
Nombre de la canción: Society
Artista: Eddie Vedder
Género: acoustic rock
Valence: 0.298
Acousticness: 0.902
Liveness: 0.111
Danceability: 0.561 
Energy: 0.327 
Instrumentalness: 0.000469 
Speechiness: 0.0327 
---
Nombre de la canción: Father And Son
Artista: Yusuf / Cat Stevens
Género: british folk
Valence: 0.371
Acousticness: 0.541
Liveness: 0.0996
Danceability: 0.

In [ ]:
playlist_name = "music video games"
playlist = search_playlist(playlist_name, access_token)

if playlist:
    print(f"Playlist encontrada: {playlist['name']} (ID: {playlist['id']})")
    # Obtener los IDs de las canciones de la playlist
    track_ids = get_track_ids_from_playlist(playlist["id"], access_token)
    track_ids = track_ids[:50]

    print("\nIDs de canciones en la playlist seleccionada:")
    print(track_ids)
else:
    print(f"No se encontró la playlist '{playlist_name}'.")

Playlist encontrada: Video Game Soundtracks (ID: 37i9dQZF1DXdfOcg1fm0VG)

IDs de canciones en la playlist seleccionada:
['285ieonEuLkll3zknYK2TY', '6DhcuZbi4FW28RLicWNmmx', '4mowfIHVIYRoepbExd1cwu', '05UMQXFCsa9oPnLgfJHVyF', '2gaZJDgE71VL9PzzUUlpMg', '66RFxA8iDSZmRqtMSsHmQq', '4Ga0jNG9Pm5g6x1whprPQd', '4NsPgRYUdHu2Q5JRNgXYU5', '4lm5D5pwWQt7kmPPoqIFAa', '3m8UfZwPMMbiWfGR8Lg2cR', '65u8xkeB9XJ40Zspr1vOJZ', '39J6EI5ysv2FmIsJHPijh2', '4prLjTDLdh6Fk2mYza9q0Y', '4xEbrGC6A3Bw7hCw3yuwJz', '6eXw5tf0oLrJRowzaOetc0', '17D3bYY9JJJjH1TUAruBJr', '7hVKcAyshAzCL16kJAdji1', '325hdqkyHBRYu6ZOZt5Yrh', '116jZYVgxrFurFTmo6pDgv', '21e8VPZa4lRxAt2Ra772f9', '19YEN3yo79SF9wO4v65XA5', '7D3owndTfR0iPyRO7sHR3g', '1Y0LhaqOcaUFxF3VBGD9Qy', '4zDOsDzhepxaCHzDiMVBEJ', '7rJEhNn1XaIXyFNNo9Qk5M', '54JLst9pXCA0BeBLcLe018', '6enLk2V2s0M6D6PGLjHxYv', '4u3Idn4PRAINifJ5lgzZDx', '7gEpP0zIY3lVwTl7I5GZmH', '1ZJevzwhAH7YHB6AFmoIqG', '0NsG1b9koK74vTJ2VJUJQU', '4BL3XnXe40ZS2YuRXWS4wX', '1m73S45l5u5Al1jQCNBVsf', '3sPz0HlbbJsiTL1jrrrx

In [ ]:
track_id =['285ieonEuLkll3zknYK2TY', '6DhcuZbi4FW28RLicWNmmx', '4mowfIHVIYRoepbExd1cwu', '05UMQXFCsa9oPnLgfJHVyF', '2gaZJDgE71VL9PzzUUlpMg', '66RFxA8iDSZmRqtMSsHmQq', '4Ga0jNG9Pm5g6x1whprPQd', '4NsPgRYUdHu2Q5JRNgXYU5', '4lm5D5pwWQt7kmPPoqIFAa', '3m8UfZwPMMbiWfGR8Lg2cR', '65u8xkeB9XJ40Zspr1vOJZ', '39J6EI5ysv2FmIsJHPijh2', '4prLjTDLdh6Fk2mYza9q0Y', '4xEbrGC6A3Bw7hCw3yuwJz', '6eXw5tf0oLrJRowzaOetc0', '17D3bYY9JJJjH1TUAruBJr', '7hVKcAyshAzCL16kJAdji1', '325hdqkyHBRYu6ZOZt5Yrh', '116jZYVgxrFurFTmo6pDgv', '21e8VPZa4lRxAt2Ra772f9', '19YEN3yo79SF9wO4v65XA5', '7D3owndTfR0iPyRO7sHR3g', '1Y0LhaqOcaUFxF3VBGD9Qy', '4zDOsDzhepxaCHzDiMVBEJ', '7rJEhNn1XaIXyFNNo9Qk5M', '54JLst9pXCA0BeBLcLe018', '6enLk2V2s0M6D6PGLjHxYv', '4u3Idn4PRAINifJ5lgzZDx', '7gEpP0zIY3lVwTl7I5GZmH', '1ZJevzwhAH7YHB6AFmoIqG', '0NsG1b9koK74vTJ2VJUJQU', '4BL3XnXe40ZS2YuRXWS4wX', '1m73S45l5u5Al1jQCNBVsf', '3sPz0HlbbJsiTL1jrrrxGT', '33eoaqosKfgXYHd6HyYmPh', '18CctEeOpjEDIAvWM5S7RE', '0AnsIh2o4sa9p8pHJOi9oI', '05CTZquJvcOZIyWlaOVBT0', '5VuHhNAOSsCxF6ltddkEUm', '6Tkn4gwyqVcpU0ecgtU3bw', '59FMS2jznmqwLYx1Noa6H4', '74RXECEEdc945gowRY4ebj', '7Aob4PraD6KWsJX7KB6gkm', '664qR3uQQOlRVIr2bTzJQz', '3gMRvptppSZGbhYBeIqxV3', '1SsqQhUJT1TxlkOXQi9XKh', '6qV7yy8DwETobwsJgTLSCe', '4p4KwzIJ6Ztjh9HsKYCgOw', '43SHYfEdVPCIag9L3xtUVm', '2lux5JzRo8SBpwjMWurAHb']
for track_id in track_ids:
    track_details = get_track_details(track_id, access_token)

    if track_details:  # Check if track_details is not None
        print(f"Nombre de la canción: {track_details['name']}")
        print(f"Artista: {track_details['artist']}")
        print(f"Género: {track_details['genre']}")
        # Accessing audio features from the track_details dictionary
        print(f"Valence: {track_details['valence']}")
        print(f"Acousticness: {track_details['acousticness']}")
        print(f"Liveness: {track_details['liveness']}")
        print(f"Danceability: {track_details['danceability']} ")
        print(f"Energy: {track_details['energy']} ")
        print(f"Instrumentalness: {track_details['instrumentalness']} ")
        print(f"Speechiness: {track_details['speechiness']}")
        print("---")
    else:
        print(f"Could not retrieve details for track ID: {track_id}")
    time.sleep(0.3)

Nombre de la canción: The Last of Us
Artista: Gustavo Santaolalla
Género: argentine ambient
Valence: 0.353
Acousticness: 0.867
Liveness: 0.098
Danceability: 0.587 
Energy: 0.467 
Instrumentalness: 0.928 
Speechiness: 0.0306
---
Nombre de la canción: Halo
Artista: Halo
Género: video game music
Valence: 0.0369
Acousticness: 0.262
Liveness: 0.0976
Danceability: 0.332 
Energy: 0.418 
Instrumentalness: 0.883 
Speechiness: 0.0439
---
Nombre de la canción: God of War Ragnarök (feat. Eivør)
Artista: Bear McCreary
Género: orchestral soundtrack
Valence: 0.0994
Acousticness: 0.555
Liveness: 0.11
Danceability: 0.234 
Energy: 0.346 
Instrumentalness: 0.744 
Speechiness: 0.0309
---
Nombre de la canción: Ezio's Family
Artista: Jesper Kyd
Género: epicore
Valence: 0.106
Acousticness: 0.81
Liveness: 0.0682
Danceability: 0.375 
Energy: 0.344 
Instrumentalness: 0.776 
Speechiness: 0.0335
---
Nombre de la canción: Dragonborn
Artista: Jeremy Soule
Género: orchestral soundtrack
Valence: 0.121
Acousticness: 0

# Creación de df

Una vez que juntamos todos los datos en un archvo .txt lo vamos a importar.
Cabe destacar que hay canciones que porel límite de la API no puedieron ser cargadas y las eliminé del archivo.

In [ ]:
import pandas as pd

# Ruta del archivo en tu entorno de Colab (asegúrate de cargarlo antes)
file_path = '/content/songs_data.txt'  # Reemplaza con la ruta real de tu archivo

# Leer el archivo y procesar los datos
data = []
with open(file_path, 'r', encoding='utf-8') as file:
    song = {}
    for line in file:
        line = line.strip()
        if line.startswith("Nombre de la canción:"):
            song["Nombre"] = line.replace("Nombre de la canción:", "").strip()
        elif line.startswith("Artista:"):
            song["Artista"] = line.replace("Artista:", "").strip()
        elif line.startswith("Género:"):
            song["Género"] = line.replace("Género:", "").strip()
        elif line.startswith("Valence:"):
            song["Valence"] = float(line.replace("Valence:", "").strip())
        elif line.startswith("Acousticness:"):
            song["Acousticness"] = float(line.replace("Acousticness:", "").strip())
        elif line.startswith("Liveness:"):
            song["Liveness"] = float(line.replace("Liveness:", "").strip())
        elif line.startswith("Danceability:"):
            song["Danceability"] = float(line.replace("Danceability:", "").strip())
        elif line.startswith("Energy:"):
            song["Energy"] = float(line.replace("Energy:", "").strip())
        elif line.startswith("Instrumentalness:"):
            song["Instrumentalness"] = float(line.replace("Instrumentalness:", "").strip())
        elif line.startswith("Speechiness:"):
            song["Speechiness"] = float(line.replace("Speechiness:", "").strip())
        elif line == "---":  # Separador entre canciones
            data.append(song)
            song = {}  # Reiniciar para la próxima canción

# Crear un DataFrame a partir de los datos procesados
df = pd.DataFrame(data)

# Mostrar el DataFrame
print(df)

# Exportar a un archivo CSV si es necesario
df.to_csv('/content/songs_features.csv', index=False)


                                                Nombre           Artista  \
0                                                 APT.              ROSÉ   
1                                     Die With A Smile         Lady Gaga   
2                                          squabble up    Kendrick Lamar   
3                                    luther (with sza)    Kendrick Lamar   
4                                    wacced out murals    Kendrick Lamar   
..                                                 ...               ...   
812                                 [WITH GREAT POWER]        Lena Raine   
813  Whatever Comes (feat. Julie Elven & Melissa R....  Oleksa Lozowchuk   
814                                          Migration  Guillaume Ferran   
815  FFVII REMAKE: 闘う者達 -なんでも屋の仕事- (Let the Battles...     Nobuo Uematsu   
816                                   Hostage - Connor     Nima Fakhrara   

                    Género  Valence  Acousticness  Liveness  Danceability  \
0         

In [ ]:
df_songs = pd.read_csv('/content/songs_features.csv')
df_songs

,Nombre,Artista,Género,Valence,Acousticness,Liveness,Danceability,Energy,Instrumentalness,Speechiness
0,APT.,ROSÉ,k-pop,0.9390,0.0283,0.3550,0.777,0.783,0.000,0.2600
1,Die With A Smile,Lady Gaga,art pop,0.5350,0.3080,0.1220,0.521,0.592,0.000,0.0304
2,squabble up,Kendrick Lamar,conscious hip hop,0.7110,0.0206,0.0783,0.874,0.672,0.000,0.1980
3,luther (with sza),Kendrick Lamar,conscious hip hop,0.5760,0.2510,0.2480,0.707,0.575,0.000,0.1250
4,wacced out murals,Kendrick Lamar,conscious hip hop,0.5020,0.2490,0.1360,0.761,0.501,0.000,0.4590
...,...,...,...,...,...,...,...,...,...,...
812,[WITH GREAT POWER],Lena Raine,indie game soundtrack,0.2170,0.7350,0.0990,0.715,0.445,0.829,0.0306
813,Whatever Comes (feat. Julie Elven & Melissa R....,Oleksa Lozowchuk,video game music,0.0824,0.1200,0.2420,0.146,0.475,0.652,0.0448
814,Migration,Guillaume Ferran,Unknown,0.4010,0.9780,0.0772,0.549,0.257,0.842,0.0335
815,FFVII REMAKE: 闘う者達 -なんでも屋の仕事- (Let the Battles...,Nobuo Uematsu,chiptune,0.3930,0.1660,0.3590,0.497,0.864,0.982,0.0516


# API Genius

Utilizamos esta api para hallar las letras de las canciones a partir del url

In [ ]:
!pip install requests oauthlib

In [ ]:
import requests
from urllib.parse import urlencode

# Credenciales de tu aplicación
client_id = "3kjRs7wtpRXVEKlQcN8eXfajMj9u_PXkf1tpQSZ4bX5dBorTwltung8bFIrZGmtA"
client_secret = "TjaQY0nEDqJRPUzwCx81MmpJpultjg1yxhzQJzrnd119fGw9gvUNiY5EDVHbYyz7N4FAo4H_ADr93jN55qSp6g"
redirect_uri = "https://oauth.pstmn.io/v1/callback"  # O el redirect URI que configuraste
scopes = "me"

# Paso 1: Generar el enlace de autorización
auth_url = "https://api.genius.com/oauth/authorize"
params = {
    "client_id": client_id,
    "redirect_uri": redirect_uri,
    "scope": scopes,
    "response_type": "code",
}

auth_link = f"{auth_url}?{urlencode(params)}"
print(f"Visita este enlace para autorizar la aplicación: {auth_link}")


Visita este enlace para autorizar la aplicación: https://api.genius.com/oauth/authorize?client_id=3kjRs7wtpRXVEKlQcN8eXfajMj9u_PXkf1tpQSZ4bX5dBorTwltung8bFIrZGmtA&redirect_uri=https%3A%2F%2Foauth.pstmn.io%2Fv1%2Fcallback&scope=me&response_type=code


In [ ]:
# Código de autorización (copiado del paso anterior)
authorization_code = "ItHN12XQaaW4HJ6722mn2oVvay5C_CSzGDwurU88L3eEHpohDx_UGzPynPWya-Za"

# Endpoint para solicitar el token
token_url = "https://api.genius.com/oauth/token"
data = {
    "grant_type": "authorization_code",
    "code": authorization_code,
    "client_id": client_id,
    "client_secret": client_secret,
    "redirect_uri": redirect_uri,
}

response = requests.post(token_url, data=data)
if response.status_code == 200:
    access_token = response.json()["access_token"]
    print("Token de acceso obtenido:", access_token)
else:
    print("Error al obtener el token:", response.json())


Token de acceso obtenido: InR5um4zdv7B252KSu6D_uK7-VG4BnvxQM2zskxdQKmro5ozf5vhyTUpdc30U-hL


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Tu API Key de Genius
GENIUS_API_KEY = 'InR5um4zdv7B252KSu6D_uK7-VG4BnvxQM2zskxdQKmro5ozf5vhyTUpdc30U-hL'

# Función para buscar la URL de las letras en Genius
def get_lyrics_url(song_title, artist):
    base_url = "https://api.genius.com/search"
    headers = {"Authorization": f"Bearer {GENIUS_API_KEY}"}
    params = {"q": f"{song_title} {artist}"}

    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        # Tomar la primera coincidencia
        hits = data['response']['hits']
        if hits:
            return hits[0]['result']['url']  # URL de las letras
    return None

# Función para extraer letras desde la URL usando BeautifulSoup
def scrape_lyrics(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            # Buscar el contenedor principal de las letras
            lyrics_div = soup.find("div", class_="Lyrics__Container-sc-1ynbvzw-6")
            if lyrics_div:
                return lyrics_div.get_text(separator="\n")
        return None
    except Exception as e:
        print(f"Error scraping lyrics from {url}: {e}")
        return None

# Procesar el DataFrame
df_songs['lyrics_url'] = df_songs.apply(lambda row: get_lyrics_url(row['Nombre'], row['Artista']), axis=1)

df_songs


,Nombre,Artista,Género,Valence,Acousticness,Liveness,Danceability,Energy,Instrumentalness,Speechiness,lyrics_url
0,APT.,ROSÉ,k-pop,0.9390,0.0283,0.3550,0.777,0.783,0.000,0.2600,https://genius.com/Rose-and-bruno-mars-apt-lyrics
1,Die With A Smile,Lady Gaga,art pop,0.5350,0.3080,0.1220,0.521,0.592,0.000,0.0304,https://genius.com/Lady-gaga-and-bruno-mars-di...
2,squabble up,Kendrick Lamar,conscious hip hop,0.7110,0.0206,0.0783,0.874,0.672,0.000,0.1980,https://genius.com/Kendrick-lamar-squabble-up-...
3,luther (with sza),Kendrick Lamar,conscious hip hop,0.5760,0.2510,0.2480,0.707,0.575,0.000,0.1250,https://genius.com/Digster-playlists-black-his...
4,wacced out murals,Kendrick Lamar,conscious hip hop,0.5020,0.2490,0.1360,0.761,0.501,0.000,0.4590,https://genius.com/Kendrick-lamar-wacced-out-m...
...,...,...,...,...,...,...,...,...,...,...,...
812,[WITH GREAT POWER],Lena Raine,indie game soundtrack,0.2170,0.7350,0.0990,0.715,0.445,0.829,0.0306,https://genius.com/Lena-raine-with-great-power...
813,Whatever Comes (feat. Julie Elven & Melissa R....,Oleksa Lozowchuk,video game music,0.0824,0.1200,0.2420,0.146,0.475,0.652,0.0448,https://genius.com/Oleksa-lozowchuk-whatever-c...
814,Migration,Guillaume Ferran,Unknown,0.4010,0.9780,0.0772,0.549,0.257,0.842,0.0335,https://genius.com/Guillaume-ferran-migration-...
815,FFVII REMAKE: 闘う者達 -なんでも屋の仕事- (Let the Battles...,Nobuo Uematsu,chiptune,0.3930,0.1660,0.3590,0.497,0.864,0.982,0.0516,None


In [ ]:
import requests

# Cargar tu DataFrame
data = {
    "Nombre": ["APT.", "Die With A Smile", "squabble up", "luther (with sza)", "wacced out murals"],
    "Artista": ["ROSÉ", "Lady Gaga", "Kendrick Lamar", "Kendrick Lamar", "Kendrick Lamar"],
    "Género": ["k-pop", "art pop", "conscious hip hop", "conscious hip hop", "conscious hip hop"],
    # Más columnas adicionales...
}
df = pd.DataFrame(data)

# Función para obtener las letras usando Lyrics.ovh
def get_lyrics(row):
    base_url = "https://api.lyrics.ovh/v1/"
    try:
        # Construir la URL de la solicitud
        response = requests.get(f"{base_url}{row['Artista']}/{row['Nombre']}")
        if response.status_code == 200:
            return response.json().get("lyrics", "Letra no encontrada")
        else:
            return "Letra no encontrada"
    except Exception as e:
        print(f"Error obteniendo la letra para {row['Nombre']}: {e}")
        return "Error"

# Aplicar la función al DataFrame
df["Lyrics"] = df.apply(get_lyrics, axis=1)

# Mostrar el DataFrame actualizado
print(df)


              Nombre         Artista             Género  \
0               APT.            ROSÉ              k-pop   
1   Die With A Smile       Lady Gaga            art pop   
2        squabble up  Kendrick Lamar  conscious hip hop   
3  luther (with sza)  Kendrick Lamar  conscious hip hop   
4  wacced out murals  Kendrick Lamar  conscious hip hop   

                                              Lyrics  
0                                Letra no encontrada  
1  (Ooh, ooh)\r\nI, I just woke up from a dream\r...  
2  God knows\r\nI am reincarnated\r\nI was starga...  
3  If this world were mine\r\nHey, Roman numeral ...  
4  Siento aquí tu presencia\r\nLa noche de anoche...  


# API Lyrics.ovh

Hallamos las letras de las canciones a través del nombre y del artista

In [ ]:
def get_lyrics(row):
    base_url = "https://api.lyrics.ovh/v1/"
    try:
        # Construir la URL de la solicitud
        response = requests.get(f"{base_url}{row['Artista']}/{row['Nombre']}")
        if response.status_code == 200:
            return response.json().get("lyrics", "Letra no encontrada")
        else:
            return "Letra no encontrada"
    except Exception as e:
        print(f"Error obteniendo la letra para {row['Nombre']}: {e}")
        return "Error"

# Aplicar la función al DataFrame
df_songs["Lyrics"] = df_songs.apply(get_lyrics, axis=1)

# Mostrar el DataFrame actualizado
df_songs.head()

,Nombre,Artista,Género,Valence,Acousticness,Liveness,Danceability,Energy,Instrumentalness,Speechiness,lyrics_url,Lyrics
0,APT.,ROSÉ,k-pop,0.939,0.0283,0.3550,0.777,0.783,0.0,0.2600,https://genius.com/Rose-and-bruno-mars-apt-lyrics,Letra no encontrada
1,Die With A Smile,Lady Gaga,art pop,0.535,0.3080,0.1220,0.521,0.592,0.0,0.0304,https://genius.com/Lady-gaga-and-bruno-mars-di...,"(Ooh, ooh)\r\nI, I just woke up from a dream\r..."
2,squabble up,Kendrick Lamar,conscious hip hop,0.711,0.0206,0.0783,0.874,0.672,0.0,0.1980,https://genius.com/Kendrick-lamar-squabble-up-...,God knows\r\nI am reincarnated\r\nI was starga...
3,luther (with sza),Kendrick Lamar,conscious hip hop,0.576,0.2510,0.2480,0.707,0.575,0.0,0.1250,https://genius.com/Digster-playlists-black-his...,"If this world were mine\r\nHey, Roman numeral ..."
4,wacced out murals,Kendrick Lamar,conscious hip hop,0.502,0.2490,0.1360,0.761,0.501,0.0,0.4590,https://genius.com/Kendrick-lamar-wacced-out-m...,Siento aquí tu presencia\r\nLa noche de anoche...


# Guardamos el resultado del df para el futuro análisis

In [ ]:
from google.colab import files

# Guardar el archivo en el entorno de Colab
df_songs.to_csv('canciones_con_letras.csv', index=False)

# Descargar el archivo CSV a tu computadora
files.download('canciones_con_letras.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>